In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys

In [2]:
name=['코코도르']
category=['별점']

# 코코도르 디퓨저 후기
ns_address="https://search.shopping.naver.com/catalog/32092966779?query=%EB%94%94%ED%93%A8%EC%A0%80&NaPm=ct%3Dlbavaolc%7Cci%3Dd568d2fe2da5f3a5a806add7855e0e6a9490426d%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Da7dc2da373ad4095734a8d9bed4c3adaecd689a1"
#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[3]/a"
category1="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[2]/a" #5점
category2="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[3]/a" #4점
category3="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[4]/a" #3점
category4="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[5]/a" #2점
category5="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[6]/a" #1점

In [3]:
#함수 선언
def add_dataframe(name,category,reviews,stars,dates,options,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star','date','option'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i],dates[i],options[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,date,option,'null','null']
        n+=1    
    return df1

In [4]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_7522/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
# 5점 리뷰 가져오기
d.find_element(By.XPATH,category1).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니라고 생각해요. 그래도 다행히 파손없이 받았습니다.
며칠전 수국세트로 구매하려다 바빠서 며칠뒤 보니 수국도 천원이 오르고 윈터옵션이 새로 있길래 그냥 윈터옵션 그린트리로 구매했어요. 사람 맘이란게 천원 오른 상태로 사려니 괜히 손해보는 느낌은 어쩔 수 없네요ㅋ ^-^;
트리는 눌려있는 상태라 제가 조심히 폈는데 원래 그런건지는 모르겠으나 솔잎이 하나는 크게 안눌렸으나 하나는 눌린게 안 펴지네요. 다른 한개도 같고요. 솔방울도 하나 떨어져있었는데 붙어있는게 아니라 철사에 끼워 걸쳐져있는거길래 제가 조심히 끼웠어요. 오래 쓸 건 아닌거 같고 올겨울 한철 잘 쓸듯요.
디퓨져에 꽂으니 예쁘긴한데 두고봐야겠지만 시간지나면 갈색테잎이 디퓨져에 젖어 풀릴듯한 생각이 드네요. 윈터옵션엔 유리관증정이 없어 수국이랑 고민하다 선택한거긴한데 차라리 유리관이 이런데 증정이면 트리모형이 손상 안되고 담에 또 쓸수 있어 더 좋았겠단 생각이 들지만, 이렇게 판매하니 만족합니다.
올겨울은 작지만 요걸로 크리스마스 느낌 내겠어요.ㅎ
전에 블랙체리 사용해봤어어 이번엔 망고 시켜봤는데 찐하고 달달한 망고향 예상했으나 생각보다는 찐하거나 망고망고한 향은 아닌거 같아요. 그래도 스틱 꽂고 향 올라오면 또 다를거 같네요. 디퓨져병에 스티커가 꽤 울게 붙어있어서 보기 싫어 심한거 하나는 떼고 집에 있던 스티커 붙이니 느낌이 또 다르고, 그린트리 하나 기존 집에 있던 다른 디퓨져에 꽂으니 또 느낌 달라요. 잘 쓰고 다음엔 다른 향으로 또 구매하겠습니다~ 평점4 20.11.11. 옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민X2+스틱/블랙X2 / 디퓨저 향선택2: 그린트리스틱X2 

2 생애 디퓨저를 처음 구매해봅니다
필요성을 못느끼다가 이번에 이사짐정리하다
방에서 자꾸 땀냄새? 묵은냄새?가 나서
인테리어소품으로도 쓸겸
리서치 끝에 유명한 코코도르 구매하게
되었네요!


18 디퓨저 뭐살까 고민하면서 찾아보고 리뷰보고 하다가 마땅한거 못찾았는데 캠핑하러 갔다가 캠핑장 화장실에서 나는 향기가 좋아서 보니까 코코도르 꺼였어요 ㅎㅎ찾아보니 가격도 저렴하고 이미 향을 알고있어서 구매하게 됐는데 배송도 빠르고 포장도 꼼꼼해서 안전하게 와서 마음에 들어요. 선물할수있게 종이백 넣어주신것도 좋구여~ 거실이랑 화장실에 하나씩 쓰는데 화장실은 좁아서 향기가 다 퍼져서 들어가자마자 향기나고 거실은 먼거리까지 다 향이 세게나진 않는데 오히려 그게 더 좋아요. 너무 센 향기맡으면 머리가 아파서 안좋아하거든요~~잘쓰고 다음에 또 재구매 하겠숨니당! 평점5 22.07.28. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

19 저 이거 두개 샀는데... 또 사도되나요? 가성비 너무 좋아요. 거실이랑 화장실에 놔뒀는데 각 방에 하나씩 은은하게 둘까 생각중이에요 최근 인터넷쇼핑 한것중에 가장 마음에 듭니다 다른 향은 안사봤지만 이건 좋습니다 원래 인위적인 향 오래 맡으면 두통이 조금 생기는게 그런것도 없어서 더 살까해요 아 뚜껑이 저렇게 열리지않고 구멍이 있었으면 더 좋을꺼같아요 또 사러 갑니다! 평점5 21.06.13. 옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

20 블랙체리랑 퓨어코튼 샀는데 향도 무난하고 딱 좋아요!
가격이나 디자인, 향 여러모로 고려했을 때 코코도르가 가장 괜찮은 것 같아요. 1+1인대도 각각 향 고를 수 있는 게 제일 좋은 것 같습니다. 장식도 서비스로 주셔서 만족했어요. 근데 라벤더 장식에 먼지같은게 너무 심하게 묻어있어요. 흐르는 물에 한 번 씻고 사용하시길 권장드립니다.. 다 쓰면 다음에도 구매할게요 👍👍 평점5 22.02.15. 선택1: 01_CO200/블랙체리 / 선택2: 03_CO200/퓨어코튼 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

21 일단 가

43 옵션으로 여러가지 선택해서 구매할 수 있는 점이 참 좋네요 전 월계꽃 크림과 핑크 비누장미 섬유스틱 디퓨저 용액 구매했어요 전에 샀던 메리고병에 꽂아봤는데 월계꽃이 길이가 긴 편이라 잘라서 크림 핑크 색상을 함께 꽂아봤더니 풍성하고 조화롭게 예쁘네요 꽃색도 은은하게 예쁘고 향긋한 디퓨저향에 집안에도 봄이 온 듯이 화사해 진 거 같아요 너무 예쁘고 만족스럽습니다 평점5 22.04.09. 선택1: 03_CO200/퓨어코튼 / 선택2: 10_CO200/로즈퍼퓸 / 데코스틱: 01_차밍로즈/다핑/미니X2+쇼핑백/소 

페이지 4 

44 기존에 사용하던 디퓨저를 거의 다 사용해서
디퓨저를 바꿔볼까 하고 찾던 중에 평이 너무 좋아서
좋은 평 믿고 주문했어요.
착한 가격에 구성도 좋은 것 같고,
집에서 사용하기도, 선물하기도 좋을 것 같아요.
듣던대로 가성비가 좋은 제품인듯요.
목요일 오후에 주문해서 오늘 받았는데,
구성품도 파손됨 없이, 빠짐없이
꼼꼼하게 잘 포장해서 배송해주셨네요.
제품안내 및 평보고, 무난해보이는 블랙체리향을
선택했는데 거실과 욕실에 놓기에 좋을 것 같아요. 평점5 21.09.04. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 01_소형/블랙체리+스틱5P / 꽃 스틱: 02_수국/베이지X2+쇼핑백/소 

45 상품 너무 맘에 들어요.
향도 제가 좋아하는 부케향이랑 꽃향인데 강하지 않고 은은해서 좋아요.
부케향은 거실에 놨는데, 은은한 향이 거실을 감싸서 너무 좋아요.
디퓨저 병도 너무 세련되고 꽃도 예뻐요.
근데 조화꽃은 따로 포장 안되서 왔어요.
그냥 덜렁 꽃만 박스에 담겨 있었는데 그래서 구겨졌어요.ㅜㅜ 그래서 엄청 손으로 핀 다음에 디퓨저에 꽂았네요~
그거 말곤 너무 대만족이에요!
담에 또 재구매 의사 200%입니다.
(꽃이 좀 더 다양했으면 좋겠어요) 평점5 22.02.10. 선택1: 06_CO200/플로랄부케 / 선택2: 07_CO200/플라워블로썸 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

46 일단 상자열

페이지 6 

72 블랙체리향 너무 좋네요~~ 너무 강한 향은 거북해서 스틱 두개 꽂아 거실에 두니 은은하게 퍼져서 좋네요~ 유칼리튜스 잎과 함께 꽂으니 인테리어 효과도 있네요^^대용량인데 초기에만 발향 좋고 금새 향이 없어진다는 후기들이 있어서 살짝 걱정은 되지만.... 한달 사용후 후기 남기겠습니다.
가성비 좋고 배송도 빨라서 좋네요!! 평점5 22.11.29. 선택1: 01_CO200/블랙체리 / 선택2: 04_CO200/프렌치라벤더 / 선택3: 04_유칼립AX2+쇼핑백/소 

73 배송이 주문하고 일주일만에 오긴 했지만 제품은 이상없이 잘 왔네요

개인적으로는 달짝지근한 블랙체리 향이 은은한 프렌치라벤더보다 나은 것 같습니다
스틱 발향도 잘돼서 방에는 블랙체리로 2개만 꽂았는데도 저한테는 진하게 나는 것 같네요. 조금 시간을 둬보고 스틱 개수를 조절해야될 것 같습니다
데코스틱도 마음에 듭니다 평점5 22.08.19. 선택1: 01_CO200/블랙체리 / 선택2: 05_CO200/프렌치라벤더 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

74 나무스틱은 금방 부러지네요!
향도 오지게 안 퍼져요! 예쓰! 예쁜 쓰레기!

다이소 가서 천원에 여러개 들어있는 섬유스틱
사와서 2-3개 꽂았는데 꽂고 나서는 발향 쩔어요

한달 지난 지금도 기름잔내 없이 향 그대로에요!
저렴해서 다른 향으로 재구매하려구요 평점5 21.03.11. 옵션 선택: 04_[1+1]새와나무 디퓨저 / 디퓨저 향: 소형/블랙체리X2 / 꽃 스틱: 우드/새와나무X2+명절깁트백(대)1P 

75 베송 포장 다 적절했구요 특가라 저렴한 가격에 꽃장식 까지 있어서 좋아요^^
편백은 기대했던더 보다 더 차분한 편백이라 쏙 맘에 들었구요
에이프릴프레쉬는 상쾌한 향인데 코로 직접 향을 맡았을때 알코올향이 좀 많이 나서 당황했으나 스틱으로 발향되는 향은 상쾌하고 괜찮았아요~ 평점5 22.11.14. 선택1: 02_CO200/에이프릴프레쉬 / 선택2: 08_CO200/편백 / 선택3: 03_수국/베이지AX

107 솔직히 수국 리드가 가까이서 보면 예쁘지는 않습니다 ㅋ
하지만 블랙 섬유 리드와 잘 어울리고, 얼굴 가까이 대고 볼 제품 아니니까 괜찮아요.
코코도르 여러 제품 사용하고 있는데, 블랙체리는 역시 날씨 추워지면 어울리네요.
한 가지 아쉬운 점은 너무 완충재 없이 제품이 배송된다는 것... 평점5 20.09.28. 옵션 스틱선택: 09_수국/진핑크X2 / 디퓨저 향선택1: 01_소형/블랙체리+스틱5PX2 / 디퓨저 향선택2: 01_소형/블랙체리+기프트백/명절 

108 선물하려고 산 건 아니지만 쇼핑백이랑 같이 오니
기분이 더 좋네용ㅋㅋ 선물용으로 구매해도 넘 좋을 것 같아요
아침에 주문하고 바로 다음날 받았구요,
깨지거나 구겨진데 없이 깔끔하게 왔어요!
블랙체리향은 양*캔들 제품으로 자주 사용해봐서
망설임 없이 선택했는데 역시나 좋네요!
욕실에 깔끔하게 두려고 유칼립투스 샀는데 넘 귀여워서
대만족입니다!ㅎㅎㅎ 평점5 21.06.17. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

109 퓨어코튼 향은 기존에 써봤던거보다.. 향이 좀 약한거 같고.. 은은하니 괜찮긴 하지만 넓은 거실에 두기에는 효과가 별로 없어요..

플로랄부케는 라일락꽃향같아요.. 화장실에 두었는데
향이 진해서 넓은 곳에 두는게 좋을거 같습니다. 평점5 22.02.28. 선택1: 03_CO200/퓨어코튼 / 선택2: 06_CO200/플로랄부케 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

110 코코도르에서 처음 주문 했어요.
제가 자주가는 곳에서 코코도르 디퓨저를 써서 향이 좋아서 구매했어요.
물건이 도착했는데, 디퓨저 스틱이 없는거에요. 다음날 코코도르에 전화하니 바로 처리해주셨어요.^^
전화하고 다음날에 웬 택배박스가 와서 뭔가 했더니 스틱이 들어있던 거였더라고요^^ (저는 스틱 보내주신다고 해서 편지봉투 사이즈에 보내주실꺼라고 예상했는데 박스여서 뭐지 했습니다^^)
지금 디퓨저에 스틱

...

1386 배 향이라고 리퓨가 달렸길래 시원할거라 생각하며 주문했어요~향 은은하고 좋아요 평점5 22.05.21. 선택1: 04_CO200/퓨어코튼 / 선택2: 04_CO200/퓨어코튼 / 데코스틱: 01_차밍로즈/다핑/미니X2+쇼핑백/소 

1387 코코도르 부모님 댁에 두었더니 향도 좋고 보기도 좋아요.
양도 많아 오래 가겠네요 평점5 22.06.01. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 01_차밍로즈/다핑/미니X2+쇼핑백/소 

1388 방안과 화장실에 두고 사용하는데 집에 들어 왔을때 좋은 향이 나서 기분이 좋네요. 평점5 21.08.13. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1389 디퓨저 종류 많지만 믿고 쌀 수 있는 코코도르.거의 다 써가는데 재구매 예정이예요 평점5 21.12.01. 디퓨저 향1: 04_소형/플라워블로썸+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 06_라벤더2PX2+쇼핑백/소 

1390 발향이 엄청나요 원룸에 하나 놓으면 향기롭습니다 다음엔 다른향으로 구매할게요! 평점5 21.07.23. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1391 저렴히 구매해서 뿌듯하네요
처음써보는거라
향이 어느정도 지속될지는
모르겠네용~~ 평점5 20.07.14. 디퓨저 향선택: 소형/블랙체리+소형/에이프릴프레쉬 / 꽃스틱 선택: 장미꽃/핑크X2 / 리드스틱 선택: 섬유스틱5P/블랙X2 

1392 엄마가 너무 좋아하시네요 향도 되게 좋아요 나중엔 다른 향도 사 봐야겠어요 모양도 정말 예쁘고 좋아요 깨진 후기가 많길래 걱정했는데 전 안전하게 왔어요 앞으로도 안전한 배송 해 주세요 평점5 21.02.14. 옵션 선택: 04_[1+1]새와나무 디퓨저 / 디퓨저 향: 소형/블랙체

1442 가격도 좋고 넘 이쁘네요^^
향도 산뜻해서 손님방에 놓을려고 구매했더니
이뻐서 제방에도 하나두고 맘에들어요
추가 구매할게요~~ 평점5 20.04.19. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 02_소형/에이프릴+스틱5P / 꽃 스틱: 07_리시안/핑크X2+쇼핑백/소 

1443 배송 빠르게 받고요 향도 너무 좋으네요 두번째구매했는데 자주 사용할 것 같아요 평점5 22.03.04. 선택1: 02_CO200/에이프릴 / 선택2: 07_CO200/플라워블로썸 / 데코스틱: 06_우드스틱/나비X2+쇼핑백/소 

1444 화장실에 나두려고 샀는데 달달포근 냄새너무 좋네요!!! 배송도 빠르고 좋습니당!! 평점5 21.05.17. 옵션 선택: 09_[1+1]수선화 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 수선화/노랑X2P+쇼핑백(소)1P 

1445 집에 들어왔을때 상쾌한 향이나서 좋고
재택중인 요즘 더욱 빛을 보는 중입니다 ㅎ 평점5 21.01.12. 옵션 스틱선택: 08_[1+1]유칼립투스 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/블랙X2 / 디퓨저 향선택2: 유칼립AX2 

1446 화장실 향기가 좋다구 하네용~~상자는 다 찢어져서 누가 반품한거 같은게 왔지만.. 평점5 22.01.04. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 07_라벤더2PX2+쇼핑백/소 

1447 향이 살짝 무거운 것 빼고는 다 만족스러워요!
구성, 가격, 배송, 포장 완벽합니다. 평점5 21.03.30. 옵션 선택: 08_[1+1]와일드로즈 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 와일드로즈/흰X2+쇼핑백(소)1P 

1448 향이 조금 연해졌나싶다가도 아침에 나가보면 잘나네요, 블랙체리 향 좋아요 달달 평점5 21.05.13. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃

1499 저렴하고 빠르고 이쁘고 향도 좋음
캔들 사려다 디퓨저 샀는데 잘한듯
이것만쓸듯 평점5 21.07.07. 옵션 선택: 06_[1+1]화이트플라워 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 화이트플라워X2+쇼핑백(소)1P 

1500 향도 좋고 꾸밀수있는것도 이뻐서 좋아요 기존 코코도르보다 향이 잘퍼지네요~!! 평점5 21.10.05. 옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 야자잎스틱X2+쇼핑백(소)1P 

1501 늘 쓰는 제품이라 쟁여두고 쓰네요. 향이 강하지 않아서 좋아요. 잘 쓰고 있어요. 평점5 21.03.31. 옵션 선택: 08_[1+1]피오니 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱/블랙X2 / 꽃 스틱: 피오니/진핑크X2+명절깁트백(대)1P 

1502 블랙체리 향이 너무 달콤하고 좋아요
다른건 아직 개봉전인데 이것도 기대할게요 평점5 22.07.07. 선택1: 01_CO200/블랙체리 / 선택2: 08_CO200/잉글리쉬 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

1503 향 확산력은 강하지 않은 것 같아요.. 방안 전체에 퍼지지는 못하는 느낌이랄까.. 평점5 22.01.17. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 01_소형/블랙체리+스틱5P / 꽃 스틱: 07_라벤더2PX2+쇼핑백/소 

페이지 95 

1504 직장에서 쓰려고 주문하면서
집들이용 선물로 두셋트 주문했는데
타사제품 쓰다가 처음으로 사봅니다
가성비는 좋은듯 하네요 평점5 22.05.09. 선택1: 01_CO200/블랙체리 / 선택2: 03_CO200/에이프릴 / 데코스틱: 06_우드스틱/나비X2+쇼핑백/소 

1505 향도 은은하니 좋구
인테리어 효과도 있어서 더 좋네용
재구매의사 완전 있습니당 평점5 21.07.10. 옵션 선택: 09_[1+1] 새와나무 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱 / 꽃 스틱: 새와나무X2+쇼핑백(소)1P 

1555 자연향을 좋아해서 편백향으로 선택했는데 향이 은은하게 퍼져서 좋은거같아요 평점5 22.08.18. 선택1: 02_CO200/편백 / 선택2: 02_CO200/편백 / 데코스틱: 04_유칼립AX2+쇼핑백/소 

1556 향도 오래가고 모양도 예뻐요~크리스마스분위기 잘 내었어요~다쓰고 또 올께요~ 평점5 21.01.12. 옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/플라워블라썸X2+스틱/블랙X2 / 디퓨저 향선택2: 그린트리스틱X2 

1557 넘 이쁘네요
향은 살짝 강하긴한데
그래두 가격비함 넘 굿굿
배송도 진짜 빨라요~ 평점5 20.09.25. 옵션 스틱선택: 03_유칼립AX2 / 디퓨저 향선택1: 04_소형/퓨어코튼+스틱5PX2 / 디퓨저 향선택2: 05_소형/플라워블로썸+기프트백/명절 

1558 방에 두려고 구매했는데

향도 은은하니 좋고(퓨어코튼향)

인테리어로도 좋아요^^ 평점5 21.12.21. 선택1: 02_CO200/에이프릴 / 선택2: 03_CO200/퓨어코튼 / 데코스틱: 03_자작나무X2+쇼핑백/소 

1559 에이프릴 프레쉬 향이 제일 좋아요 ! 그리고 별이 빛나는 ~ 요것도 아주 좋네요 ! 평점5 22.10.10. 선택1: 03_CO200/에이프릴 / 선택2: 03_CO200/에이프릴 / 데코스틱: 04_유칼립AX2+쇼핑백/소 

1560 항상 쓰던 향이라서 믿고 구매했어요 `
코코도르는 진짜 가성비 끝판인것 같아요 평점5 21.02.02. 옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+명절깁트백(대)1P 

1561 냄새가 강하지 않고 은은해서 좋아요. 다 쓰고 나면 다른 향도 구매해 보려구요 평점5 21.09.03. 옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱/블랙X2 / 꽃 스틱: 야자잎스틱X2+쇼핑백(소)1P 

1562 블랙체리는 별로ᆢ달달한 향일거란 예상과는 달리 접착제 냄새ᆢ라

1612 친구 선물로 두개 주문했습니다 향이 생각보다 강하지 않아 당황스러웠지만 은은하게 나는 향도 괜찮은거 같습니다 배송은 빠른편은 아니지만 포장은 잘 되어서 왔습니다 평점5 22.01.15. 선택1: 04_CO200/프렌치라벤더 / 선택2: 01_CO200/블랙체리 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

1613 항상 여기꺼만 사서 쓰는데
향도 너무 좋고 너무 강렬한 향이아니라 은은하게 잘퍼져서 너무 좋아요!!
그리구 인테리어 효과까지 ㅎㅎㅎ 잘쓸게요 평점5 21.07.01. 옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

1614 지인집에서 향이 너무 좋길래
물어보고 구매했어요
은은하게 풍기는향 넘 좋네요
모양도 고급스럽고 인테리어도 한몫하네요
기분 넘 좋네요^^ 평점5 21.12.21. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 03_자작나무X2+쇼핑백/소 

1615 집들이 선물로 가져갔는데 센스있다고 칭찬받았어요 ㅎㅎ 향도 좋고 최고네요 평점5 20.12.10. 옵션 스틱선택: 07_탑송이/핑크/3PX2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼+기프트백/명절 

1616 가성비 최고예요 다들 달달한 냄새난다고 그래요 ㅋㅋ 다써가서 이필사려구요 평점5 21.08.28. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

1617 향이 처음에는 퍼지고 좋아요. 3/2 정도 남으면 좀 은은해요. 라벤더 참 이뻐요 평점5 21.09.28. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

1618 향이 예전에 산 것보다 많이 진해진 것 같은 건 기분탓일까요~꽃장식은 예

In [6]:
# 5점 리뷰 데이터프레임으로 표현
df1=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df1.head()

,type,category,review,star,date,option
1,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민...
2,코코도르,별점,생애 디퓨저를 처음 구매해봅니다\n필요성을 못느끼다가 이번에 이사짐정리하다\n방에서...,평점5,21.03.20.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...
3,코코도르,별점,딸아이 친구가 이사가면서 딸냄을 초대했어요. 그래도 이사 후 초대받은거여서 그냥 보...,평점5,21.06.03.,옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...
4,코코도르,별점,향이 나지 않습니다. 디퓨저를 사용하는 가장 중요한 목적인 '향' 부분이 빵점에 가...,평점2,21.03.20.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...
5,코코도르,별점,생애 디퓨저를 처음 구매해봅니다\n필요성을 못느끼다가 이번에 이사짐정리하다\n방에서...,평점5,21.06.03.,옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...


In [7]:
# 크롤링한 5점 데이터의 갯수
len(df1)

1631

In [8]:
# 5점 데이터 전처리

df1 = df1.drop_duplicates()
df1 = df1.reset_index(drop=True)
df1_1 = df1['option'].str.split('/',expand=True)
df1 = pd.concat([df1,df1_1],axis=1)
df1.head()

,type,category,review,star,date,option,0,1,2,3,4,5,6,7,8
0,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민...,옵션 스틱선택: 06_[1+1]그린트리 디퓨저,디퓨저 향선택1: 소형,망고쟈스민X2+스틱,블랙X2,디퓨저 향선택2: 그린트리스틱X2,None,None,None,None
1,코코도르,별점,생애 디퓨저를 처음 구매해봅니다\n필요성을 못느끼다가 이번에 이사짐정리하다\n방에서...,평점5,21.03.20.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...,옵션 선택: 06_[1+1]장미 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 장미꽃,핑크2겹기본X2+쇼핑백(소)1P,None,None,None
2,코코도르,별점,딸아이 친구가 이사가면서 딸냄을 초대했어요. 그래도 이사 후 초대받은거여서 그냥 보...,평점5,21.06.03.,옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...,옵션 선택: 01_[1+1]라벤더 디퓨저,디퓨저 향: 소형,퓨어코튼X2+스틱,블랙X2,꽃 스틱: 라벤더,퍼플,2pX2+쇼핑백(소)1P,None,None
3,코코도르,별점,향이 나지 않습니다. 디퓨저를 사용하는 가장 중요한 목적인 '향' 부분이 빵점에 가...,평점2,21.03.20.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...,옵션 선택: 06_[1+1]장미 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 장미꽃,핑크2겹기본X2+쇼핑백(소)1P,None,None,None
4,코코도르,별점,생애 디퓨저를 처음 구매해봅니다\n필요성을 못느끼다가 이번에 이사짐정리하다\n방에서...,평점5,21.06.03.,옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...,옵션 선택: 01_[1+1]라벤더 디퓨저,디퓨저 향: 소형,퓨어코튼X2+스틱,블랙X2,꽃 스틱: 라벤더,퍼플,2pX2+쇼핑백(소)1P,None,None


In [9]:
# 전처리 후 크롤링한 5점 데이터의 갯수
len(df1)

1621

In [10]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_7522/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
# 4점 리뷰 가져오기
d.find_element(By.XPATH,category2).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니라고 생각해요. 그래도 다행히 파손없이 받았습니다.
며칠전 수국세트로 구매하려다 바빠서 며칠뒤 보니 수국도 천원이 오르고 윈터옵션이 새로 있길래 그냥 윈터옵션 그린트리로 구매했어요. 사람 맘이란게 천원 오른 상태로 사려니 괜히 손해보는 느낌은 어쩔 수 없네요ㅋ ^-^;
트리는 눌려있는 상태라 제가 조심히 폈는데 원래 그런건지는 모르겠으나 솔잎이 하나는 크게 안눌렸으나 하나는 눌린게 안 펴지네요. 다른 한개도 같고요. 솔방울도 하나 떨어져있었는데 붙어있는게 아니라 철사에 끼워 걸쳐져있는거길래 제가 조심히 끼웠어요. 오래 쓸 건 아닌거 같고 올겨울 한철 잘 쓸듯요.
디퓨져에 꽂으니 예쁘긴한데 두고봐야겠지만 시간지나면 갈색테잎이 디퓨져에 젖어 풀릴듯한 생각이 드네요. 윈터옵션엔 유리관증정이 없어 수국이랑 고민하다 선택한거긴한데 차라리 유리관이 이런데 증정이면 트리모형이 손상 안되고 담에 또 쓸수 있어 더 좋았겠단 생각이 들지만, 이렇게 판매하니 만족합니다.
올겨울은 작지만 요걸로 크리스마스 느낌 내겠어요.ㅎ
전에 블랙체리 사용해봤어어 이번엔 망고 시켜봤는데 찐하고 달달한 망고향 예상했으나 생각보다는 찐하거나 망고망고한 향은 아닌거 같아요. 그래도 스틱 꽂고 향 올라오면 또 다를거 같네요. 디퓨져병에 스티커가 꽤 울게 붙어있어서 보기 싫어 심한거 하나는 떼고 집에 있던 스티커 붙이니 느낌이 또 다르고, 그린트리 하나 기존 집에 있던 다른 디퓨져에 꽂으니 또 느낌 달라요. 잘 쓰고 다음엔 다른 향으로 또 구매하겠습니다~ 평점4 20.11.11. 옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민X2+스틱/블랙X2 / 디퓨저 향선택2: 그린트리스틱X2 

2 배송은 이틀정도 걸렸으니 빨랐던 것 같아요!
일단 ❣️에이프릴 프레쉬❣️향 진짜 강추에요!!!
제가 카*** 선물하기에서도 사람들 후기보면서 향기 고르는데 엄청 고민 많이 했고,
어디 

23 추가구입한 수국 리드가 꽃이 없는 상품으로 와서 컴플레인하고 교환요청했는데 정상제품 2개로 보내주셨어요(컴플레인한 상품은 1개). 그러실 필요 없었는데 감사히 받았습니다. 유리관도 누락돼서 같이 보내주셨어요. 향이 꽤 강해서 스틱 하나만 꽂았더니 적당하네요. 평점4 22.04.01. 선택1: 01_CO200/블랙체리 / 선택2: 03_CO200/퓨어코튼 / 데코스틱: 01_차밍로즈/다핑/미니X2+쇼핑백/소 

24 제가 늘 사용하던 코코도르 디퓨저~!!
좋은 가격에 구매 했네요~~^^
에이프릴 프레쉬향은 첨인데 달콤한 향이라
제 취향은 아니예요~~ㅎ
그리고 액체가 뿌연데 이상있는건 아니겠죠~?? 늘사용하는 코코도른데 이런 상태는 첨이라요~~암튼 구매한거라 잘 사용하겠습니다~~♡♡ 평점4 21.08.05. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

25 제가 쓸제품이 아니라 아직 개봉전인데 다른제품 코코피치는 좋은데 잉글리쉬페어프리는 영 아니예요~
리프레싱에어와 플라워블로썸은 향은 제발 둘다 좋은
향이기를 바랄께요~
오배송으로 좀 시간이 걸리긴 했는데 담부터는
착오 없이 배송 부탁드립니다~ 평점4 22.01.09. 선택1: 05_CO200/리프레싱에어 / 선택2: 07_CO200/플라워블로썸 / 데코스틱: 05_수국/베이지AX2+쇼핑백/소 

26 배송 엄청 오래걸렸어요 ㅠㅠ 택배 파업때문이라고 연락 주셔서 아쉽지만 기다렸습니다
선물용이라서 상자나 쇼핑백은 너무 좋았어요
파손된 부분도 없었어요! 근데 같이 오는 꽃이 아무리 모아도 벌어져서 살짝 아쉬웠습니다
그래도 상품은 잘 배송되니까 선물용으로
아주 좋을것 같아요 평점4 22.03.11. 선택1: 04_CO200/프렌치라벤더 / 선택2: 05_CO200/리프레싱에어 / 데코스틱: 01_차밍로즈/다핑/미니X2+쇼핑백/소 

27 빠른배송 / 빠른 교환 / 좋은 상품

배송은 넘 빨라서 좋네요.
다만 처음에 왔던 

66 화장실 디퓨져로 사용중인데 잉글리쉬는 꽃향기가 강해서 개인적으로는 블랙체리가 무난한것 같네용!! 하나사면 3~5개월은 쓰는 것 같아요!! 평점4 22.04.18. 선택1: 01_CO200/블랙체리 / 선택2: 08_CO200/잉글리쉬 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

67 향도 너무 좋고 인테리어 효과도 좋아요. 여러번 구매했는데 이번엔 배송실수가 있었지만 빠르게 대처 해주셔서 정상적으로 잘 받았습니다. 평점4 20.09.30. 옵션 스틱선택: 10_몬스테라AX2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 02_소형/에이프릴프레쉬+기프트백/명절 

68 향도 잘 퍼지고 양이 많아서 좋긴한데 처음 사보는 향이라 그런지 제가 좋아하는 타입은 아니라서 다음에는 다른 향을 구입해 보고 싶네요. 평점4 21.12.06. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 05_소형/망고쟈스민+스틱5P / 꽃 스틱: 02_수국/베이지X2+쇼핑백/소 

69 엄 처음에 열었을때 향이 원래 알던 블랙체리랑 좀 다른 것 같아 뭐지 싶었어요 좀 쓰다보면 괜찮아지겟져 같이 온 소품도 나름 갬성잇어오 평점4 20.11.19. 옵션 스틱선택: 08_[1+1]유칼립투스 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/블랙X2 / 디퓨저 향선택2: 유칼립AX2 

70 에어프릴 프레쉬가 상쾌하고 좋아요~
생각보다 향이 강해서 스틱을 1-2개 정도만 꽂고 사용하다 취향에 따라 추가하는게 좋을 것 같습니다. 평점4 20.11.07. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 01_소형/블랙체리 

71 향이 좋아요! 추천받은 향으로 구매했는데 정말 마음에 들어요

스틱 3개+ 장식 총 4개 펼쳐놨는데 향 강하면 하나 빼려구요
거실 + 작은 방에 두었습니다. 집에 들어 오자마자 좋은 향이 났음 해요~ 평점4 21.01.22. 옵션 스틱선택: 

111 지난번보다 금액이 조금 올라서 별하나 뺐어요..
향은 지난번과 같은 에어프릴향.. 괜찮아요..
지난번에는 유칼립투스 스틱으로 선택해서,,, 완전 실망였었는데..이번 라벤더 스틱은 괜찮네요..
다음번에는 가격이 안올랐으면 하는 마음이네요.. 평점4 21.01.14. 옵션 스틱선택: 07_[1+1]라벤더 디퓨저 / 디퓨저 향선택1: 소형/에이프릴X2+스틱/블랙X2 / 디퓨저 향선택2: 라벤더/퍼플/2pX2 

페이지 13 

112 선물용으로 대량 주문했는데 다 뽁뽁이 포장에 전용 상자에 담겨 빠른 배송으로 도착했지만 그 중 하나 케이스 상단 우측 부분이 찢어져서 왔네요 ㅠㅠ

이번에는 번거로워서 그냥 테이브 부착해서 선물할 예정이지만 제품 포장시 좀더 주의 부탁드리겠습니다!! 기타 옵션 상품과 쇼핑백은 퀄리티 모두 굿굿~~~ 평점4 22.10.08. 선택1: 05_CO200/프렌치라벤더 / 선택2: 05_CO200/프렌치라벤더 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

113 박스 사진은 못 찍었는데
박스에 송장 붙이고 그 위에 테이프 붙여 보내주셔서 송장 제거하는게 꽤 힘들었네요. 요새 송장에 있는 개인정보 유출로 문제가 많은데 이렇게 보내주심 제거하기 힘들어요.. 참고하셔서 개선되면 좋겠어요.

하나는 거실에 놨는데 몬스테라가 자꾸 빙글 돌아가서 뒷모습으로 돌아가네요.. 평점4 21.04.15. 옵션 선택: 05_[1+1]몬스테라 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 몬스테라스틱AX2+쇼핑백(소)1P 

114 용기 디자인이 심플하고 고급스러워요.에이프릴 스프링 사용 중인데 스틱을 라벤더 두개 포함해서 10개 꽂아서 사용중인데 향이 약하네요.그리고 아이들 있는 가정을 위해 쿤달 제품처럼 플라스틱(페트) 용기로도 판매했으면 좋겠어요 평점4 21.06.18. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

115 제

149 사용 시작하고 며칠간은 향도 잘 퍼지고 좋았는데요, 한 일주일 지나니 거의 향이 안나더라구요. 그래서 스틱도 뒤집어 높고 해봐도 향이 약하네요. 너무 휘발성이 강한가봐요. 지속력이 너무 약합니다 평점4 22.02.13. 선택1: 03_CO200/퓨어코튼 / 선택2: 02_CO200/에이프릴 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

150 저렴하게 디퓨져와 함께 크리스마스 장식품도 구매한 것 같네요. 그런데 저 사슴 부분이 견고하지가 못해서 아쉬워요ㅠ 금방 떨어져서 지금일단 테이프로 묶어뒀어요. 평점4 21.12.19. 디퓨저 향1: 03_소형/퓨어코튼+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 01_루돌프X2+목도리/초록X2+쇼핑백/소 

151 일단 1+1이라 싸게 샀어요. 근데 제가 생각했던 코튼향이랑은 조금 다른거같아요^^; 암튼 발향은 잘됩니다. 스틱 하나랑 꽃만 해놨는데도 꽤 퍼져요. 여러개하면 약간 머리 아플듯합니다. 평점4 21.03.07. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

페이지 15 

152 제품은 100점 만점으로 만족합니다. \n다만 완충재가 너무 부실해서 박스 안에서 이리저리 굴러다니면서 파손위험이 있는 포장상태가 아쉽네요. \n차라리 선물박스에 넣은채로 배송되는게 더 안전할거 같아요 평점4 21.02.20. 옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+명절깁트백(대)1P 

페이지 16 

153 향 \고민하다가 그냥 무난하게 퓨어코튼 하자 했는데 생각보다 인위적인 향이라 놀랬어여 근데 그 막대를 1개만 꽂으니까 은은하니 괜찮은 거같아여 다음엔 다른 향으로 사봐야겠어여 평점4 21.05.17. 선택1: 01_CO200/블랙체리 / 선택2: 08_CO200/잉글리쉬 / 데코스틱: 01_차밍로즈/다핑/미니X2+쇼핑

...

1222 배송도 빠르고 향기도너무좋네요 또 구매할께요 평점4 22.01.22. 선택1: 04_CO200/프렌치라벤더 / 선택2: 04_CO200/프렌치라벤더 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

1223 향도 좋고 이뻐요~ 하지만 향이 강하진 않네요ㅠ 평점4 21.03.24. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1224 스틱많이꼽으면 금방날아갈것같아서요

향좋아요 평점4 21.03.15. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1225 생각했던 거 보다 많이 단 향인데 나쁘지 않아요 평점4 21.03.21. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1226 향이 좀 약한듯
화장실 두곳창가에 넣고 사용 중 평점4 20.12.29. 옵션 스틱선택: 09_[1+1]수국 디퓨저 / 디퓨저 향선택1: 소형/플라워블라썸X2+스틱/블랙X2 / 디퓨저 향선택2: 수국/베이지AX2 

1227 제가 생각하는 향기는 아니였지만 잘쓰겠읍니다 평점4 21.06.21. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

1228 배송도 빠르고 향을 바로 느낄수 있어서 좋아요. 평점4 22.11.18. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 선택3: 02_라벤더2PX2+쇼핑백/소 

1229 생각보다 발향이 잘 되진 않는 점이 좀 아쉬워용 평점4 21.07.06. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

페이지 83 

1230 은은한 향이 오래가면 좋은데ㅡ

1288 구생탱이에다가 놓으면 향기 은은하고 좋아요~ 평점4 20.12.24. 옵션 스틱선택: 01_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/블랙X2 / 디퓨저 향선택2: 그린트리스틱X2 

1289 현관에 놓고 사용하니 은은하게 냄새가 좋아요 평점4 21.04.08. 옵션 선택: 04_[1+1]새와나무 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2 / 꽃 스틱: 우드/새와나무X2+쇼핑백(소)1P 

페이지 86 

1290 배송은 조금 아쉬웠지만, 가성비로 최고입니다. 평점4 21.08.08. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1291 향은 괜찮은데 저 꽃들은..좀 그렇더라구요..^^; 평점4 20.11.02. 옵션 스틱선택: 07_탑송이/핑크/3PX2 / 디퓨저 향선택1: 05_소형/플라워블로썸+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼 

페이지 87 

1292 은은하니 좋습니다
항상 리필해서 쓰고 있네요~ 평점4 22.02.21. 선택1: 04_CO200/프렌치라벤더 / 선택2: 08_CO200/잉글리쉬 / 데코스틱: 01_루돌프X2+목도리/초록X2+쇼핑백/소 

1293 잘받았습니다
장미는 부러져서 사용하기는 좀.. 평점4 20.03.26. 디퓨저 향선택: 소형/블랙체리X2 / 꽃스틱 선택: 장미꽃/핑크X2 / 리드스틱 선택: 섬유스틱5P/블랙X2 

1294 항상쓰는 코코도르
근데 조화 티가 너무나요ㅎ 평점4 20.11.02. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 04_소형/퓨어코튼+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼 

1295 배송도 빠르고 향기도 좋고 인테리어 효과까지 평점4 21.10.04. 디퓨저 향1: 04_소형/플라워블로썸+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 08_국화/파스텔블루X2+쇼핑백/소 

1296 향이 조금

1354 디퓨저는 코코도르!
향도 고를수 잇고 가격도 착하고
가성비 좋아요! 온 집안에 향기 가득해서
기분이 좋아요 평점4 21.10.08. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 05_소형/망고쟈스민+스틱5P / 꽃 스틱: 06_라벤더2PX2+쇼핑백/소 

1355 예쁜데 향기는 진하지않고 은은하게 나네요 평점4 21.05.26. 옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

1356 제품 포장에 먼지가 많았지만 향은 좋았어요 평점4 21.04.05. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

1357 향도 은은하고 사무실에 놓고 쓰기에 좋네요 평점4 21.01.11. 옵션 스틱선택: 09_[1+1]수국 디퓨저 / 디퓨저 향선택1: 소형/퓨어코튼X2+스틱/블랙X2 / 디퓨저 향선택2: 수국/베이지AX2 

1358 향이 진하고 좋은데 너무 금방 줄어들어요ㅠ 평점4 22.01.22. 디퓨저 향1: 01_CO200/블랙체리 / 디퓨저 향2: 01_CO200/블랙체리 / 꽃 스틱: 05_수국/베이지AX2+쇼핑백/소 

1359 안방엔스틱한개만꽂아도 은은하게 향좋네요 평점4 20.10.27. 옵션 스틱선택: 03_유칼립AX2 / 디퓨저 향선택1: 01_소형/블랙체리+스틱5PX2 / 디퓨저 향선택2: 02_소형/에이프릴프레쉬+기프트백/명절 

1360 배송 너무 빠르고 원하는 향이네요 좋습니다 평점4 22.11.14. 선택1: 05_CO200/화이트머스크 / 선택2: 05_CO200/화이트머스크 / 선택3: 03_수국/베이지AX2+쇼핑백/소 

1361 배송빠르고 좋은데 병에 센건지 묻어 있네요 평점4 20.02.05. 디퓨저 향선택: 소형/에이프릴프레쉬X2 / 꽃스틱 선택: 장미꽃/핑크X2 / 리드스틱 선택: 섬유스틱5P/블랙X2 

1362 만족 합니

1423 가성비 좋아요! 원룸에는 스틱을 두개만 꽂아야될것같아요 너무향이강해서 ㅠㅠ 평점4 20.12.14. 옵션 스틱선택: 06_[1+1]유칼립투스 디퓨저 / 디퓨저 향선택1: 소형/퓨어코튼X2+스틱/블랙X2 / 디퓨저 향선택2: 유칼립AX2 

1424 가격대비 좋아요. 늘 사용하던거고 근데 대용량이없네요
근데 로즈향은 별로인듯 평점4 22.10.16. 선택1: 01_CO200/블랙체리 / 선택2: 10_CO200/로즈퍼퓸 / 데코스틱: 01_차밍로즈X2+쇼핑백/소 

1425 이사와서 고여있는 집냄새와 화장실 냄새때문에 구입했어요
잎은 좀...
향은 좋아요 평점4 21.04.23. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

페이지 96 

1426 향 좋아요.
앞으로도 많이 이용하겠습니다.^^ 평점4 21.05.13. 옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

1427 선물로 구매햇는데 너무 좋아하셧어요~향도 왜케 좋냐며 뿌듯햇어요!!많이 파세요~~ 평점4 21.11.14. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 05_소형/망고쟈스민+스틱5P / 꽃 스틱: 03_비누장미/소/진핑크X4+쇼핑백/소 

1428 인기많은 향이길래 사봤는데 그냥 무난한 향인듯.. 하지만 저는 다른향이 더 좋아요.. 평점4 21.02.17. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+명절기프트백(대)1P 

1429 냄새가.잘나고 아주좋아여 ㅎㅎ많이파세요 평점4 21.07.20. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

1430 열린 방문으로 은은한 향이 나와서 좋아요. 평점4 22.02.23. 

1491 냄새너무좋아요 재구매했는데 역시네요ㅋ 평점4 22.07.04. 선택1: 02_CO200/편백 / 선택2: 05_CO200/프렌치라벤더 / 데코스틱: 06_우드스틱/나비X2+쇼핑백/소 

1492 가성비 짱!!!
장미꽃이 있어 고급져보여요^^ 평점4 20.08.05. 디퓨저 향선택: 소형/블랙체리+소형/망고쟈스민 / 꽃스틱 선택: 01_장미꽃/핑크X2 / 리드스틱 선택: 섬유스틱5P/블랙X2 

1493 물품 잘 수령했어요~디자인이 심플하네요~ 평점4 21.03.11. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1494 예쁘긴한데 향이 좀 더 강했으면 좋겠어요 평점4 20.11.28. 옵션 스틱선택: 07_탑송이/핑크/3PX2 / 디퓨저 향선택1: 01_소형/블랙체리+스틱5PX2 / 디퓨저 향선택2: 02_소형/에이프릴프레쉬+기프트백/명절 

페이지 100 

페이지 101 

페이지 102 

1495 블랙체리는 대중적인 향인거 같네요.
좋아요~ 평점4 21.12.05. 디퓨저 향1: 01_CO200/블랙체리 / 디퓨저 향2: 02_CO200/에이프릴 / 꽃 스틱: 06_목화솜X2+쇼핑백/소 

1496 양도 많고 향도 좋은데 저렴하기까지 좋아요 평점4 22.03.17. 선택1: 03_CO200/퓨어코튼 / 선택2: 11_CO200/화이트머스크 / 데코스틱: 05_목화솜X2+쇼핑백/소 

1497 코코도르는 향이 강해서 좋아요~ 많이파세요 평점4 21.11.11. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 04_비누장미/소/연핑크X4+쇼핑백/소 

1498 향이 너무 좋네요 선물용으로도 괜찮겠어요 평점4 22.02.15. 선택1: 02_CO200/에이프릴 / 선택2: 03_CO200/퓨어코튼 / 데코스틱: 03_수국/베이지AX2+명절/기박 

1499 화장실에 둘려고 샀는데 은은한 향이 좋네

In [12]:
# 4점 리뷰 데이터프레임으로 표현
df2=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df2.head()

,type,category,review,star,date,option
1,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민...
2,코코도르,별점,배송은 이틀정도 걸렸으니 빨랐던 것 같아요!\n일단 ❣️에이프릴 프레쉬❣️향 진짜 ...,평점5,21.02.25.,옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱...
3,코코도르,별점,구성도 깔끔하고 배송도 잘 와서 좋았어요! 원래 향수나 디퓨져 안쓰던 집이라 그런가...,평점4,21.03.20.,옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...
4,코코도르,별점,평소 사무실과 방에 화사한 향기를 주고 싶어서 고민하던 차에 긍정적인 후기가 많아서...,평점4,22.02.18.,선택1: 03_CO200/퓨어코튼 / 선택2: 06_CO200/플로랄부케 / 데코스...
5,코코도르,별점,향 너무 좋아요! 거실에 두니 지나다닐 때마다 은은하니 기분 좋네요. 소형 디퓨저인...,평점4,21.10.03.,옵션 선택: 03_[1+1]몬스테라 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/...


In [13]:
# 크롤링한 4점 데이터의 갯수
len(df2)

1539

In [14]:
# 4점 데이터 전처리

df2 = df2.drop_duplicates()
df2 = df2.reset_index(drop=True)
df2_2 = df2['option'].str.split('/',expand=True)
df2 = pd.concat([df2,df2_2],axis=1)
df2.head()

,type,category,review,star,date,option,0,1,2,3,4,5,6,7,8
0,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민...,옵션 스틱선택: 06_[1+1]그린트리 디퓨저,디퓨저 향선택1: 소형,망고쟈스민X2+스틱,블랙X2,디퓨저 향선택2: 그린트리스틱X2,None,None,None,None
1,코코도르,별점,배송은 이틀정도 걸렸으니 빨랐던 것 같아요!\n일단 ❣️에이프릴 프레쉬❣️향 진짜 ...,평점5,21.02.25.,옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱...,옵션 선택: 02_[1+1]유칼립투스 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 유칼립AX2+쇼핑백(소)1P,None,None,None,None
2,코코도르,별점,구성도 깔끔하고 배송도 잘 와서 좋았어요! 원래 향수나 디퓨져 안쓰던 집이라 그런가...,평점4,21.03.20.,옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...,옵션 선택: 03_[1+1]수국 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 수국,베이지AX2+쇼핑백(소)1P,None,None,None
3,코코도르,별점,평소 사무실과 방에 화사한 향기를 주고 싶어서 고민하던 차에 긍정적인 후기가 많아서...,평점4,22.02.18.,선택1: 03_CO200/퓨어코튼 / 선택2: 06_CO200/플로랄부케 / 데코스...,선택1: 03_CO200,퓨어코튼,선택2: 06_CO200,플로랄부케,데코스틱: 03_수국,베이지AX2+쇼핑백,소,None,None
4,코코도르,별점,향 너무 좋아요! 거실에 두니 지나다닐 때마다 은은하니 기분 좋네요. 소형 디퓨저인...,평점4,21.10.03.,옵션 선택: 03_[1+1]몬스테라 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/...,옵션 선택: 03_[1+1]몬스테라 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 몬스테라스틱X2+쇼핑백(소)1P,None,None,None,None


In [15]:
# 전처리 후 크롤링한 4점 데이터의 갯수
len(df2)

1526

In [16]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_7522/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
# 3점 리뷰 가져오기
d.find_element(By.XPATH,category3).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

페이지 2 

1 배송도 빠르고 향도 좋았으나
고객응대가 너무 고무적이라
답답하고 질문에 같은답만 계속..
짜증도 좀 났엇구요..상품 중 수국은 다 찌그러졌고라벤더는 S자 로 휘어져서 왓네요 조금만 신경쓰면 될텐데
굳이 휘어짐없는 디퓨저는 제품크기의 약 2.5배 가량의 종이케이스에..종이케이스를 살짝 줄이고 그 남는 공간에 꽃을 그냥 넣어주기만하면 공간이 비좁아 휘어지고 찌그러질 일은 없을텐데
처음 주문할때부터 답변응하는 사람이 융통성이 없더니 배송까지 융통성없네요 코코도르 나온지가 꽤 오래됏는데
제품을 이렇게 다루다니..
디퓨저에 꽂으니 안 꽂는게 나을듯요
다른 디퓨저브랜드는 장식용꽃 을 굉장히 소중히 다뤄주던데..향도 중요하지만 보이는것도 중요하니까요
좀 실망햇네요 평점3 22.03.18. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

2 선물받은 코코도르가 향이 정말 좋아서 그 기분 공유하고 싶어서 지인 선물용으로 구매를 했는데 포장박스가 찢어져있고 쇼핑백은 바닥 면이 사진처럼 되어있어서 바닥면 종이를 뒤집으려하니 이미 저 상태로 접착되어 있었어요 😭 선물용이라 열어보지 않았는데 후기를 보니 혹시나 누락이 있을까 걱정스럽네요😅
수국도 전혀 포장이 안되어서 눌려있던데 다른 분들 후기를 보니 한송이씩 비닐캡 싸주셨네요?ㅠㅠ
쇼핑백 손잡이는 원래 매듭을 직접 하는건지?
포장상태가 쪼끔 아쉽습니다 평점3 22.09.26. 선택1: 03_CO200/에이프릴 / 선택2: 03_CO200/에이프릴 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

3 일단 꽃이 화면처럼 예쁘지 않아요 많~이 다이소느낌이예요 멀리서 보면 괜찮아요
발향이 잘되지 않고 (리드 다꽃았어요) 향이 다소 실망스럽네요 유명한 제품이라 기대했는데 저렴한 향이랄까... 약간 멀미나요 그리고 플로랄향? 그건 카페화장실 가면 나는 향이예요 그만큼 이 향이 인기가 있어서 많이 사용하시겠지만 거실에서카페 화장실향이 

37 향은 괜찮아요
조화가 찌그러진건지 꽃줄기가 구부러져서 꽂아도 이쁘지 않네요 그냥 빼버렸어요
그리고 배송이 아주 느렸습니다 평점3 22.03.25. 선택1: 05_CO200/리프레싱에어 / 선택2: 06_CO200/플로랄부케 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

38 다 좋은데 선물용으로 샀는데 상자가 뜯어뎌 있어서 좀 그렇네요.. 분명 포장하시면서 보셨을텐데^^
그래도 다른건 다 좋습니다. 평점3 22.01.14. 선택1: 02_CO200/에이프릴 / 선택2: 05_CO200/리프레싱에어 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

39 리프레싱헤어는 비누향이고 시트러스라벤더는 싫어하는 남자스킨향이네요 향을 맡아볼수없어 열심히 리뷰보고 구입한건데 걍 기존에 쓰던 걸로 주문할걸 그랬네요 그래도 리프레싱헤어향은 괜찮아서 다행입니다 평점3 22.01.20. 선택1: 05_CO200/리프레싱에어 / 선택2: 09_CO200/시트러스라벤더 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

40 상품은 만족합니다
추석기프트백이라해서
샘프방향제줄알았더니
포장박스 가왔네요
저희는 필요없는 재활용휴지통으로 갑니다
굳이 이런 멘트로 안하셨음좋겠습니다
포장박스라고 하세요!! 평점3 20.09.11. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 02_소형/에이프릴프레쉬+기프트백/명절 

41 향은 상품설명서에 쓰여져 있는것처럼 참 좋은데... 꽃이 완전 찌그러져서 왔어요ㅠㅠ 꽃이 포인트인데...고것만 좀 신경써주시면 감사하겠습니다. 평점3 21.04.07. 옵션 선택: 07_[1+1]피오니 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 피오니/소핑크X2+쇼핑백(소)1P 

42 제품은 괜찮은데 포장 배송이 좀 안좋네요. 선물하기 위해서 샀는데 제품 상자가 찢어져 있어서 난감합니다. 개별 제품에 충격흡수비닐을 싸매주셨으면 좋겠어요. 평점3 22.01.28. 

89 가격도 저렴하고 배송도 빠르고 좋은데요. 향이 좀 약해요 근처에 가야만 향기가 나요ㅠ 평점3 21.10.27. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 01_동백꽃/옐로우X2+쇼핑백/소 

90 생각보다 향이 약해요. 좋게 말하면 은은하구요...스틱을 3개 넣었는데도 많이 약하네요. 평점3 21.01.25. 옵션 스틱선택: 05_[1+1]유칼립투스 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/블랙X2 / 디퓨저 향선택2: 유칼립AX2 

91 목화 스틱이 뿌러져서 왔어요..
선물하려했는데 포장백도 없고..해서 그냥 제가 쓰렵니다 평점3 20.12.09. 옵션 스틱선택: 03_[1+1]목화솜 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/블랙X2 / 디퓨저 향선택2: 목화솜/조화X2 

92 향이 잘안나서 스틱을 추가로 계속 꼽게 되네요. 예전에 쓰던 제품이랑 다른 느낌이에요 평점3 21.04.25. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

93 선물용으로 산건데 포장이 미흡한거 같아요~박스 하나가 파손되어와서 좀 그렇네요ㅜㅜ 평점3 22.06.18. 선택1: 01_CO200/블랙체리 / 선택2: 03_CO200/에이프릴 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

94 블랙체리향은 처음에만 좀 진하게 나는거 같아요.
다른향들은 근처만가도 은은하니 향이 나는데 블랙체리는 별루 안나더라구요.
일단 2개 해뒀는데 어떨지 확인해 봐야겠어요 평점3 21.07.28. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

95 가성비 좋습니다~~ 제가 코가 막혀서 향기는 잘 모르겠지만 데코용으로 마음에 들어여^^ 평점3 22.03.30. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200

141 코코도르 저번에 구매했을땐 괜찮았던것같은데 이번껀 발향이 크게되는것같진 않아요ㅡㅡ;; 다만 욕실에두니 인테리어 효관 있네요..
수국이 자꾸 돌아가는것만 빼면..ㅎ 평점3 21.05.18. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

142 향좋아요 올리브영서 향맡아보고 무난한향이라서 구매했어요?너무향수같은향이랑 베이비파우더향 싫어하는데 이건 비누향+달콤한향?그런향이나용 가격이비싼편이긴하지만 암튼좋내용 평점3 21.03.26. 옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크2겹기본X2+쇼핑백(소)1P 

143 병은 4개인데 쇼핑백은 2개라 쇼빙백구매는 별도로 해야할꺼같아요~
꽃이랑 스틱도 같이 디퓨저 상자에넣을수있게 해주심 좋겠네요^^ 평점3 21.05.11. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

144 설 연휴가 껴있었음에도 배송이 빨라서 좋았어요\n냄새도 마음에 들고요\n다만 라벤더 꽃 스틱에 먼지같은데 많이 묻어있어서 떨어지는게 좀 그렇네요.... 평점3 21.02.15. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+명절기프트백(대)1P 

145 향이 많이 나지 않아요.,.가까이 가서 맡아야 향기가 나는듯ㅠ 유리관의 용도는 모르지만 깨져서 왔어요..상자에 유리조각이 돌아다니고 손 다칠뻔했네요 평점3 20.12.10. 옵션 스틱선택: 07_[1+1]몬스테라 디퓨저 / 디퓨저 향선택1: 소형/에이프릴X2+스틱/블랙X2 / 디퓨저 향선택2: 몬스테라스틱AX2 

146 상품이 나눠져서 왔는데 그에 대한 공지를 전혀 받지 못했어요
수량은 맞았지만 디퓨저가 포장된 박스가 구겨진 경우도 많았습니다 평점3 21.05.22.

189 총3가지향중 에이프릴? 젤 머리아파요 원뿔이라 하나는 누굴선물주던지 해야해요 꽃시장은 젤괜찮네요 평점3 21.09.04. 옵션 선택: 05_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지X2+쇼핑백(소)1P 

페이지 16 

190 클린코튼은 생각보다 발향이 진해서 별로였는데 쓰다보니 적응되어 괜찮아여.... 다음엔 다른향을 사야할꺼 같아요. 평점3 21.12.28. 선택1: 03_CO200/퓨어코튼 / 선택2: 10_CO200/로즈퍼퓸 / 데코스틱: 06_목화솜X2+쇼핑백/소 

191 양이 많아서 아직도 방에 향이 나네요. 고급스러운 향은 아니지만 이제 코가 익숙해져서 은은한듯 합니다 평점3 21.05.21. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

192 첫 개봉이라서 발향은 좋은데 나중에는 사용해봐야 알거같아요! 무난한 체리향 했는데 생각하던 향은 아니고 머리가 좀 아파요ㅡ 평점3 22.06.30. 선택1: 01_CO200/블랙체리 / 선택2: 12_CO200/화이트머스크 / 데코스틱: 05_화이트플라워X2+쇼핑백/소 

193 양 많고 겨울인테리어도 되서 좋은데, 발향은 강하지않은것 같아요.. 기존쓰던거랑 2개 놓고 쓰고있어요. 평점3 20.12.31. 옵션 스틱선택: 01_[1+1]윈터 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/화이트X2 / 디퓨저 향선택2: 솔방울스틱X2+눈꽃/화이트X2 

194 상품은잘받았어요. 유리스틱이 완전 가루가 되어왔네요. 뽁뽁이로 감싸있어서 뭔지모르고 풀었다가 유리가루폭탄이 ㅠㅠㅠ 포장할때 더 신경쓰셔야겠어요. 향은 괜찮은것같네요 평점3 21.08.30. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

195 배송은 3ㅡ4일정도 걸렸어요
향은 약하다는 의견들

...

페이지 59 

871 디퓨저 향이 좋습니다 평점3 22.12.03. 선택1: 05_CO200/프렌치라벤더 / 선택2: 07_CO200/플로랄부케 / 데코스틱: 06_우드스틱/나비X2+쇼핑백/소 

872 잘사용하고 있음 ㅎㅎ 평점3 21.08.26. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

873 향이 별오 안나요ㅠㅠ 평점3 21.10.25. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

874 가격대비 가성비 좋음 평점3 20.09.20. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 05_소형/플라워블로썸+스틱5PX2 / 디퓨저 향선택2: 02_소형/에이프릴프레쉬+기프트백/명절 

875 잘사용해보겠습니다.. 평점3 22.06.21. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 05_화이트플라워X2+쇼핑백/소 

876 향이 좋아서 만족해요 평점3 20.11.29. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 01_소형/블랙체리+스틱5PX2 / 디퓨저 향선택2: 05_소형/플라워블로썸+기프트백/명절 

877 너무 달아요 향이ㅠㅠ 평점3 21.06.23. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

878 생각보다 향은 약해요 평점3 21.08.30. 옵션 선택: 05_[1+1]수국 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지X2+쇼핑백(소)1P 

879 용량도 괜찮고 좋아요 평점3 21.03.04. 옵션 선택: 08_[1+1]피오니 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 피오니/진핑크X2+쇼핑백(소)1P 

880 발

948 흠.. 향이 그닥 좋은지 모르겠어요ㅠㅠ 향이 크게 느껴지지도 않구용.. 평점3 21.07.31. 옵션 선택: 09_[1+1] 새와나무 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱 / 꽃 스틱: 새와나무X2+쇼핑백(소)1P 

949 그냥 그런대로 ... 평점3 21.09.03. 옵션 선택: 09_[1+1] 새와나무 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱 / 꽃 스틱: 새와나무X2+쇼핑백(소)1P 

950 디퓨져는 양이 줄지않았는데 향은 다 날라갔는지 별로 나지않네요 평점3 20.12.26. 옵션 스틱선택: 01_[1+1]윈터 디퓨저 / 디퓨저 향선택1: 소형/플라워블라썸X2+스틱/화이트X2 / 디퓨저 향선택2: 솔방울스틱X2+눈꽃/화이트X2 

951 늘쓰던 향인데 이번에 향이 바뀌었나요? 기존에 쓰던것과 미묘하게 다른 향이 나는 것 같아요~ 평점3 21.09.08. 옵션 선택: 05_[1+1]수국 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지X2+쇼핑백(소)1P 

952 수고하세요 ~~~~~~ 평점3 21.03.19. 옵션 선택: 04_[1+1]새와나무 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2 / 꽃 스틱: 우드/새와나무X2+쇼핑백(소)1P 

953 향이 조아요...... 평점3 21.01.11. 옵션 스틱선택: 01_[1+1]윈터 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/화이트X2 / 디퓨저 향선택2: 솔방울스틱X2+눈꽃/화이트X2 

954 섬유유연제 향 기대하고 구매했는데 너무 기대가 컸던거같네요. 스틱 1개로 사용하는데도 향이 강하네요 평점3 21.01.09. 옵션 스틱선택: 10_[1+1]장미 디퓨저 / 디퓨저 향선택1: 소형/에이프릴X2+스틱/블랙X2 / 디퓨저 향선택2: 장미꽃/핑크X2 

955 그저그래요.. ~~ 평점3 21.03.31. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1009 3가지 향을 샀은데 한개는 향을 잘못 골랐는지
별로에요 평점3 20.08.25. 꽃스틱 선택: 05_피오니/소/핑크X2 / 디퓨저 향선택: 소형/블랙체리+소형/플라워블로썸 / 리드스틱 선택: 섬유스틱5P/블랙X2 

1010 화장실에 하수구 냄새같은게 나서 샀는데 발향이 약하네요ㅠㅠ 아쉽습니다 향은 너무 좋은데ㅠㅠ 다섯개 다 꼿아쓰는데도 아쉬워요 평점3 21.05.31. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

1011 생각했던향이 아니고 향이강해서 제게는 안맞어요 평점3 21.11.22. 옵션 선택: 07_[1+1]장미 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

1012 향은 너무 좋은데
발향이 너무약해서 화장실에 놨어요 평점3 21.07.14. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

1013 향이 좋았어요 그런데 향이 오래 가지는 못 하는거 같아요 평점3 22.01.27. 선택1: 05_CO200/리프레싱에어 / 선택2: 01_CO200/블랙체리 / 데코스틱: 03_자작나무X2+쇼핑백/소 

1014 향이 생각보다 오래 가지 않습니다
좀 향이 더 오래 갔으면하는 아쉬움이 남네요 평점3 20.11.19. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 04_소형/퓨어코튼+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼 

1015 검은색 스틱이랑 유칼립투스 조화 같이 왔어요 근데 위쪽 잎이 뜯겨져 왔어요ㅋ 평점3 20.12.19. 옵션 스틱선택: 05_[1+1]유칼립투스 디퓨저 / 디퓨저 향선택1: 소형/에이프릴X2+스틱/블랙X2 / 디퓨저 향선택2: 유칼립AX2 

1016 제가 생각했던 향이 아니네요... 그리고 향이 잘 안나요ㅠ 평점3 21.05.13. 옵션 선택: 01

1071 개인적으로 향 별로네요 ......ㅠㅠㅠ 평점3 21.01.24. 옵션 스틱선택: 09_[1+1]수국 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2+스틱/블랙X2 / 디퓨저 향선택2: 수국/베이지AX2 

1072 가격도좋고향도좋습니다 평점3 21.03.23. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+명절깁트백(대)1P 

1073 좋아요자주자주시킬게요 평점3 21.08.31. 옵션 선택: 09_[1+1] 새와나무 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱 / 꽃 스틱: 새와나무X2+쇼핑백(소)1P 

1074 감사합니다잘받았습니다 평점3 20.09.12. 옵션 스틱선택: 04_다알리아/소/화이트X2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼+기프트백/명절 

1075 좋아요또구매할거같아요 평점3 22.08.03. 선택1: 01_CO200/블랙체리 / 선택2: 03_CO200/에이프릴 / 데코스틱: 04_유칼립AX2+쇼핑백/소 

1076 빠른배송감사드려요ㅎㅎ 평점3 21.02.02. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+명절기프트백(대)1P 

1077 빠른배송너무좋 습ㄴ다. 평점3 21.06.21. 옵션 선택: 08_[1+1]비누장미/소/디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 비누장미/소/진핑크2PX2+쇼핑백(소)1P 

1078 영롱하고이뻐요 글고 1+1에 핵득템한기분
빠른배송도감사합니당^^!!한달써보공후기갈게요 감사합니다 평점3 22.06.04. 선택1: 05_CO200/프렌치라벤더 / 선택2: 01_CO200/블랙체리 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

1079 냄세가 너무 머리아파요 ㅠㅠㅠ 평점3 20.12.16. 옵션 스틱선택: 03_[1+1]목화솜 디퓨저 / 디퓨

1137 Jdjdjjdjejjdjfjjwkkdkckkwkwkkdkfk 평점3 21.08.14. 옵션 선택: 07_[1+1]장미 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

1138 향이약해요 향이약해요 평점3 21.07.07. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

1139 많이파세요 많이파세요 평점3 20.06.01. 디퓨저 향선택: 소형/퓨어코튼X2 / 꽃스틱 선택: 장미꽃/핑크X2 / 리드스틱 선택: 섬유스틱5P/블랙X2 

1140 좋아요 좋아여 좋아요 평점3 21.10.28. 디퓨저 향1: 03_소형/퓨어코튼+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 02_수국/베이지X2+쇼핑백/소 

1141 그냥저냥.그냥저냥... 평점3 20.06.01. 디퓨저 향선택: 소형/플라워블로썸X2 / 꽃스틱 선택: 장미꽃/핑크X2 / 리드스틱 선택: 섬유스틱5P/블랙X2 

1142 괜찮아요. 괜찮아요 평점3 21.11.16. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 03_소형/퓨어코튼+스틱5P / 꽃 스틱: 08_국화/파스텔블루X2+쇼핑백/소 

1143 좋아용좋아요좋아요좋아요 평점3 20.11.06. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 01_소형/블랙체리+스틱5PX2 / 디퓨저 향선택2: 05_소형/플라워블로썸 

1144 잘쓸ㅋ베요잘쓸께요잘쓸께요잘쓸께요 평점3 22.11.17. 선택1: 05_CO200/화이트머스크 / 선택2: 06_CO200/화이트쟈스민 / 선택3: 02_라벤더2PX2+쇼핑백/소 

1145 빠른 배송 빠른 배송 평점3 20.09.16. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼+기프트백/명절 

페이지 75 

1

In [18]:
# 3점 리뷰 데이터프레임으로 표현
df3=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df3.head()

,type,category,review,star,date,option
1,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱...
2,코코도르,별점,배송도 빠르고 향도 좋았으나\n고객응대가 너무 고무적이라\n답답하고 질문에 같은답만...,평점3,22.03.18.,선택1: 03_CO200/에이프릴 / 선택2: 03_CO200/에이프릴 / 데코스틱...
3,코코도르,별점,선물받은 코코도르가 향이 정말 좋아서 그 기분 공유하고 싶어서 지인 선물용으로 구매...,평점3,22.09.26.,옵션 스틱선택: 02_장미꽃/핑크X2 / 디퓨저 향선택1: 05_소형/플라워블로썸+...
4,코코도르,별점,일단 꽃이 화면처럼 예쁘지 않아요 많~이 다이소느낌이예요 멀리서 보면 괜찮아요\n발...,평점3,20.10.05.,옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱...
5,코코도르,별점,괜찮을거같아 주문해봤는데 향이 생각보다 무지 쎄요 머리아플정도.. 원래 디퓨저가 스...,평점3,21.08.06.,옵션 선택: 03_[1+1]몬스테라 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/...


In [19]:
# 전처리 후 크롤링한 3점 데이터의 갯수
len(df3)

1156

In [20]:
# 3점 데이터 전처리

df3 = df3.drop_duplicates()
df3 = df3.reset_index(drop=True)
df3_3 = df3['option'].str.split('/',expand=True)
df3 = pd.concat([df3,df3_3],axis=1)
df3.head()

,type,category,review,star,date,option,0,1,2,3,4,5,6,7,8
0,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱...,선택1: 01_CO200,블랙체리,선택2: 01_CO200,블랙체리,데코스틱: 03_수국,베이지AX2+쇼핑백,소,None,None
1,코코도르,별점,배송도 빠르고 향도 좋았으나\n고객응대가 너무 고무적이라\n답답하고 질문에 같은답만...,평점3,22.03.18.,선택1: 03_CO200/에이프릴 / 선택2: 03_CO200/에이프릴 / 데코스틱...,선택1: 03_CO200,에이프릴,선택2: 03_CO200,에이프릴,데코스틱: 03_수국,베이지AX2+쇼핑백,소,None,None
2,코코도르,별점,선물받은 코코도르가 향이 정말 좋아서 그 기분 공유하고 싶어서 지인 선물용으로 구매...,평점3,22.09.26.,옵션 스틱선택: 02_장미꽃/핑크X2 / 디퓨저 향선택1: 05_소형/플라워블로썸+...,옵션 스틱선택: 02_장미꽃,핑크X2,디퓨저 향선택1: 05_소형,플라워블로썸+스틱5PX2,디퓨저 향선택2: 03_소형,망고쟈스민+기프트백,명절,None,None
3,코코도르,별점,일단 꽃이 화면처럼 예쁘지 않아요 많~이 다이소느낌이예요 멀리서 보면 괜찮아요\n발...,평점3,20.10.05.,옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱...,옵션 선택: 02_[1+1]유칼립투스 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 유칼립AX2+쇼핑백(소)1P,None,None,None,None
4,코코도르,별점,괜찮을거같아 주문해봤는데 향이 생각보다 무지 쎄요 머리아플정도.. 원래 디퓨저가 스...,평점3,21.08.06.,옵션 선택: 03_[1+1]몬스테라 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/...,옵션 선택: 03_[1+1]몬스테라 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 몬스테라스틱X2+쇼핑백(소)1P,None,None,None,None


In [21]:
# 전처리 후 크롤링한 3점 데이터의 갯수
len(df3)

1155

In [22]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_7522/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
# 2점 리뷰 가져오기
d.find_element(By.XPATH,category4).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

페이지 2 

1 향이 나지 않습니다. 디퓨저를 사용하는 가장 중요한 목적인 '향' 부분이 빵점에 가깝습니다. 후기사진에서 보듯 스틱을 절대 적게 넣은 것이 아님에도 코를 디퓨저에 박고 맡아야 살짝 납니다. (사진속 투명 유리병과 담긴 리필액 모두 코코도르 제품입니다) 또한, 배송 전 상의 없이 재고 소진이란 이유로 제가 택한 디자인과 다른 제품이 왔다는 점 또한 매우 실망스럽습니다. 특히 그냥 바꿔 보내신 우드스틱에서 사우나? 훈제향이 나서 더욱 디퓨저 본래의 향이 나지 않네요. 우드스틱은 크리스마스 시즌 제품을 보내신 건가요?
____________________________________________
1차 후기 이후 며칠 동안 사용해보며 추가 리뷰 남깁니다.
먼저 향은 계속 미미하나 첫 날보다는 조금씩 올라오는게 느껴집니다.
교환 환불 다 귀찮은 우드스틱의 훈제향도 조금씩 빠지는 것 같네요.
다만 모양은 볼때마다 아쉬움 뚝뚝...

탁월한 발향 제품은 아니지만 가격을 생각하여 별점 하나 추가합니다. 평점2 20.12.26. 옵션 스틱선택: 01_[1+1]새와나무 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민X2 / 디퓨저 향선택2: 우드스틱/새와나무X2 

2 별로였어요...여러번 주문했는데 이번 향은...멀미가 나는 느끼한 향이고 포장 상자도 찌그러져 있었어요..선물용 쇼핑백이 있었는데 누구 선물이었으면 큰ㄹ날뻔 했네요.. 조화도 대충 넣으신건지.. 찌그러져서 꽃과 입이 꺽여있더라구요.. 봄이라 화사하라고 선택했는데 너무 아쉬워요.. 서로 엮어서 대충 모양 잡아놨네요..그래서 두개 샀지만 하나만 꺼내놨네요..다른 하나는 언제 꺼낼수 있을지 모르겠네요-.-;; 평점2 22.04.02. 선택1: 02_CO200/에이프릴 / 선택2: 02_CO200/에이프릴 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

3 전에 머스크향 디퓨저를 샀는데 향이 너무 약해서 다른걸로 구매해봤습니다. 프렌치라벤더는 제 방에 뒀는데 처음에는 발향이 잘 되는가 

46 정품 코코도르가 맛는지요? 향이 넘 약하고 좀 아쉽네요. 평점2 21.07.18. 옵션 선택: 09_[1+1] 새와나무 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱 / 꽃 스틱: 새와나무X2+쇼핑백(소)1P 

47 배송 진짜 느리네요
완충제도 밑에 하나만 깔고..박스도 다 찌그려지고 안 깨진게 다행이네요 평점2 22.03.28. 선택1: 02_CO200/에이프릴 / 선택2: 02_CO200/에이프릴 / 데코스틱: 02_라벤더2PX2+쇼핑백/소 

48 조아요 근데포장상태는쫌 안조아요
선물할꺼엿는데 박스가다찍으러져서와서ᆢ
ㅡㅡ속상합니다 포장을 신경써서 보내주시면
조아쓸꺼 같아요 평점2 20.09.18. 옵션 스틱선택: 02_장미꽃/핑크X2 / 디퓨저 향선택1: 01_소형/블랙체리+스틱5PX2 / 디퓨저 향선택2: 02_소형/에이프릴프레쉬+기프트백/명절 

페이지 6 

49 늘 쓰던 제품 구입했는데 발향이 좀 약하고 겉포장뿐아니라 속포장도 다 찌그러지고 찢어져서 왔어요 ㅠㅠ 평점2 21.06.10. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

50 가격적인부분은 좋은데 향이 별루에요ㅠㅠ
너무 독해엽 평점2 20.10.08. 옵션 스틱선택: 03_유칼립AX2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼+기프트백/명절 

51 재구매인데 포장 상태 엉망임.. 깨져서왔으면 어쩔뻔... 평점2 21.01.07. 옵션 스틱선택: 10_[1+1]장미 디퓨저 / 디퓨저 향선택1: 소형/퓨어코튼X2+스틱/블랙X2 / 디퓨저 향선택2: 장미꽃/핑크X2 

52 누가 반품한걸 보냈나..검수도 안하는지..박스가 뜯은 흔적이 아니라 누가 다 찢어논걸 보냄.. 평점2 21.12.02. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 07_라벤더2PX2+쇼핑백/소

114 향은 좋은데 발향이 잘 안되요 평점2 21.05.25. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

115 그냥 그래요
,,,,,,,,,
,,,,,,
,,,,,,,, 평점2 21.04.07. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+쇼핑백(소)1P 

116 향기가 좀 인위적이에요ㅜㅜ 평점2 21.06.09. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

117 질리는 재질의 향...싼티나요 평점2 21.01.03. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 05_소형/플라워블로썸+스틱5PX2 / 디퓨저 향선택2: 02_소형/에이프릴프레쉬 

118 향이 약하네요..완전 가까이서 맡아야 냄새나요.
이번향은 좀 실망이네요.. 평점2 21.07.19. 옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 야자잎스틱X2+쇼핑백(소)1P 

119 향이 잘 안나요 좀 아쉬워요 평점2 21.07.09. 옵션 선택: 04_[1+1]새와나무 디퓨저 / 디퓨저 향: 소형/블랙체리X2 / 꽃 스틱: 우드/새와나무X2+쇼핑백(소)1P 

120 냄새가이상해요 머리아파요 평점2 21.07.14. 옵션 선택: 07_[1+1]장미 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

121 하루향기나고 냄새가안나요 평점2 21.10.30. 옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 야자잎스틱X2+쇼핑백(소)1P 

122 우드스틱이 깨져서 왔네요.. 평점2 21.05.08. 옵션 선택: 04_[1+1]새와나무 디퓨저 / 디퓨저 

177 가격이 저렴한 것과 모형꽃임을 감안하더라도 좀... 너무 그저그래요 평점2 21.10.28. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 02_소형/에이프릴+스틱5P / 꽃 스틱: 01_동백꽃/옐로우X2+쇼핑백/소 

178 가격이 저렴한 것과 모형꽃임을 알고 구매하였지만 너무 그저그래요 평점2 21.10.28. 디퓨저 향1: 03_소형/퓨어코튼+스틱/5P / 디퓨저 향2: 03_소형/퓨어코튼+스틱5P / 꽃 스틱: 02_수국/베이지X2+쇼핑백/소 

179 개인적으로 향도진하고가까이가면눈이 따끔거리는것같아 아쉬워요 평점2 20.10.09. 옵션 스틱선택: 07_탑송이/핑크/3PX2 / 디퓨저 향선택1: 05_소형/플라워블로썸+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼+기프트백/명절 

180 향이 디퓨져 둔 곳에서만 나요..퍼지지를 않아요 집이 크지도 않은데 평점2 21.07.23. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

181 향 좋고 원플원이라 샀는데 배송진짜 느리네요 선물하려했는데 너무 늦게와서 그냥 둘다 제가 써요 평점2 22.03.22. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 04_유칼립AX2+쇼핑백/소 

182 생각했던 퓨어코튼향이 아니였어요.
많이 싼티나는 향이랄까요.. ㅠㅠ
향도 쎄서 다른 향이랑 섞어서 사용해볼까해요. 평점2 21.08.13. 옵션 선택: 07_[1+1]장미 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

183 향이... 아쉽네요 머리아파요
맡아보고 사야하나봅니다 ㅠㅠ 평점2 21.09.04. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

184 처음엔 냄새가 잘 나다가 한 달 지나니깐 향이

245 넘 실망이에요. 포장도 엉망이고 계속 쓰던 제품인데 발향이 넘 약해요 ㅠㅠ 평점2 21.06.10. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

246 별로입ㄴㅣ다 재구매 노노 평점2 21.06.30. 옵션 선택: 09_[1+1]수선화 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 수선화/노랑X2P+쇼핑백(소)1P 

247 발향이 잘 안되는거같아요 평점2 21.09.20. 옵션 선택: 05_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지X2+쇼핑백(소)1P 

248 그냥 값어치만큼향이네요 평점2 21.02.04. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+명절깁트백(대)1P 

페이지 18 

249 유리관이 하나 깨져서 왔네요... 평점2 20.09.26. 옵션 스틱선택: 03_유칼립AX2 / 디퓨저 향선택1: 02_소형/에이프릴프레쉬+스틱5PX2 / 디퓨저 향선택2: 04_소형/퓨어코튼+기프트백/명절 

페이지 19 

250 아쉽네요 여러모로.. 평점2 22.09.27. 선택1: 03_CO200/에이프릴 / 선택2: 06_CO200/리프레싱에어 / 데코스틱: 05_화이트플라워X2+쇼핑백/소 

251 잘받았습니다 ~~~~~~ 3 평점2 20.10.15. 꽃스틱 선택: 01_장미꽃/핑크X2 / 디퓨저 향선택: 소형/플라워블로썸X2 / 리드스틱 선택: 섬유스틱5P/화이트X2 

252 처음에만 향이 나는거같고 시간이지나고는 향이많이 떨어지네요 평점2 22.03.17. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 01_루돌프X2+목도리/초록X2+쇼핑백/소 

253 향이 맘에 안드네요 평점2 22.11.01. 선택1: 05_CO200/프렌치라벤더 / 선택2: 07_CO

In [24]:
# 2점 리뷰 데이터프레임으로 표현
df4=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df4.head()

,type,category,review,star,date,option
1,코코도르,별점,향이 나지 않습니다. 디퓨저를 사용하는 가장 중요한 목적인 '향' 부분이 빵점에 가...,평점4,20.12.26.,옵션 스틱선택: 01_[1+1]새와나무 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민...
2,코코도르,별점,별로였어요...여러번 주문했는데 이번 향은...멀미가 나는 느끼한 향이고 포장 상자...,평점2,22.04.02.,선택1: 02_CO200/에이프릴 / 선택2: 02_CO200/에이프릴 / 데코스틱...
3,코코도르,별점,전에 머스크향 디퓨저를 샀는데 향이 너무 약해서 다른걸로 구매해봤습니다. 프렌치라벤...,평점2,22.02.16.,선택1: 04_CO200/프렌치라벤더 / 선택2: 05_CO200/리프레싱에어 / ...
4,코코도르,별점,향이란 개인차가 심한거 같네요~~리뷰가 좋아서 구입했는데 퓨어코튼향은 저하고 안 맞...,평점2,20.07.31.,디퓨저 향선택: 소형/퓨어코튼X2 / 꽃스틱 선택: 01_장미꽃/핑크X2 / 리드스...
5,코코도르,별점,발향 진짜 너무 너무 약해요ㅠㅠ 같이 구매한 에이프릴이랑 똑같이 스틱 3개 꽂았는데...,평점2,21.10.03.,옵션 선택: 05_[1+1]수국 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙...


In [25]:
# 크롤링한 2점 데이터의 갯수
len(df4)

289

In [26]:
# 2점 데이터 전처리

df4 = df4.drop_duplicates()
df4 = df4.reset_index(drop=True)
df4_4 = df4['option'].str.split('/',expand=True)
df4 = pd.concat([df4,df4_4],axis=1)
df4.head()

,type,category,review,star,date,option,0,1,2,3,4,5,6,7,8
0,코코도르,별점,향이 나지 않습니다. 디퓨저를 사용하는 가장 중요한 목적인 '향' 부분이 빵점에 가...,평점4,20.12.26.,옵션 스틱선택: 01_[1+1]새와나무 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민...,옵션 스틱선택: 01_[1+1]새와나무 디퓨저,디퓨저 향선택1: 소형,망고쟈스민X2,디퓨저 향선택2: 우드스틱,새와나무X2,None,None,None,None
1,코코도르,별점,별로였어요...여러번 주문했는데 이번 향은...멀미가 나는 느끼한 향이고 포장 상자...,평점2,22.04.02.,선택1: 02_CO200/에이프릴 / 선택2: 02_CO200/에이프릴 / 데코스틱...,선택1: 02_CO200,에이프릴,선택2: 02_CO200,에이프릴,데코스틱: 03_수국,베이지AX2+쇼핑백,소,None,None
2,코코도르,별점,전에 머스크향 디퓨저를 샀는데 향이 너무 약해서 다른걸로 구매해봤습니다. 프렌치라벤...,평점2,22.02.16.,선택1: 04_CO200/프렌치라벤더 / 선택2: 05_CO200/리프레싱에어 / ...,선택1: 04_CO200,프렌치라벤더,선택2: 05_CO200,리프레싱에어,데코스틱: 05_수국,베이지AX2+쇼핑백,소,None,None
3,코코도르,별점,향이란 개인차가 심한거 같네요~~리뷰가 좋아서 구입했는데 퓨어코튼향은 저하고 안 맞...,평점2,20.07.31.,디퓨저 향선택: 소형/퓨어코튼X2 / 꽃스틱 선택: 01_장미꽃/핑크X2 / 리드스...,디퓨저 향선택: 소형,퓨어코튼X2,꽃스틱 선택: 01_장미꽃,핑크X2,리드스틱 선택: 섬유스틱5P,블랙X2,None,None,None
4,코코도르,별점,발향 진짜 너무 너무 약해요ㅠㅠ 같이 구매한 에이프릴이랑 똑같이 스틱 3개 꽂았는데...,평점2,21.10.03.,옵션 선택: 05_[1+1]수국 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙...,옵션 선택: 05_[1+1]수국 디퓨저,디퓨저 향: 소형,퓨어코튼X2+스틱,블랙X2,꽃 스틱: 수국,베이지X2+쇼핑백(소)1P,None,None,None


In [27]:
# 전처리 후 크롤링한 2점 데이터의 갯수
len(df4)

249

In [28]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_7522/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [29]:
# 1점 리뷰 가져오기
d.find_element(By.XPATH,category5).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니라고 생각해요. 그래도 다행히 파손없이 받았습니다.
며칠전 수국세트로 구매하려다 바빠서 며칠뒤 보니 수국도 천원이 오르고 윈터옵션이 새로 있길래 그냥 윈터옵션 그린트리로 구매했어요. 사람 맘이란게 천원 오른 상태로 사려니 괜히 손해보는 느낌은 어쩔 수 없네요ㅋ ^-^;
트리는 눌려있는 상태라 제가 조심히 폈는데 원래 그런건지는 모르겠으나 솔잎이 하나는 크게 안눌렸으나 하나는 눌린게 안 펴지네요. 다른 한개도 같고요. 솔방울도 하나 떨어져있었는데 붙어있는게 아니라 철사에 끼워 걸쳐져있는거길래 제가 조심히 끼웠어요. 오래 쓸 건 아닌거 같고 올겨울 한철 잘 쓸듯요.
디퓨져에 꽂으니 예쁘긴한데 두고봐야겠지만 시간지나면 갈색테잎이 디퓨져에 젖어 풀릴듯한 생각이 드네요. 윈터옵션엔 유리관증정이 없어 수국이랑 고민하다 선택한거긴한데 차라리 유리관이 이런데 증정이면 트리모형이 손상 안되고 담에 또 쓸수 있어 더 좋았겠단 생각이 들지만, 이렇게 판매하니 만족합니다.
올겨울은 작지만 요걸로 크리스마스 느낌 내겠어요.ㅎ
전에 블랙체리 사용해봤어어 이번엔 망고 시켜봤는데 찐하고 달달한 망고향 예상했으나 생각보다는 찐하거나 망고망고한 향은 아닌거 같아요. 그래도 스틱 꽂고 향 올라오면 또 다를거 같네요. 디퓨져병에 스티커가 꽤 울게 붙어있어서 보기 싫어 심한거 하나는 떼고 집에 있던 스티커 붙이니 느낌이 또 다르고, 그린트리 하나 기존 집에 있던 다른 디퓨져에 꽂으니 또 느낌 달라요. 잘 쓰고 다음엔 다른 향으로 또 구매하겠습니다~ 평점4 20.11.11. 선택1: 04_CO200/프렌치라벤더 / 선택2: 10_CO200/로즈퍼퓸 / 데코스틱: 01_루돌프X2+목도리/초록X2+쇼핑백/소 

2 별 1개도아깝네요 배송 이따위로보내고 소비자 기만합니까??!! 재고 떨이하기에 급급해서 소비자가 상품 배송받고 어떤 기분일지, 이걸 설선물로 줄려고 기다리고있다가 얼마나 난감할지는 생각도안했죠

36 제가 전에 사용했던 향을 똑같이 신청했는데 향이 틀리고 알콜냄새가 너무 많이 나네요. ..
향이 달라진것 같아요. .
블랙체리 달콤한향이 진한걸로 알고 있는데 이번에 구입한건 달콤향보단 알콜향이 많이 심하네요~~
조심 실망스럽네요~~ 평점1 22.01.02. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 05_수국/베이지AX2+쇼핑백/소 

37 외관상 예뻐요. 근데 향이 안나요.
10평 남짓한 방안과 2평 남짓한 화장실에 하나 씩 두었어요. 디퓨저 양이 줄어드는건 눈에 보여요. 근데 향이 안나요. 냄새에 민감한 편인데, 향이 하나도 안 나요.
은은한 향? 그런 것도 없어요. 그냥 말 그대로 향이 없어요. 디퓨저에 코 박으면 납니다. 근데 그런 용도가 아니잖아요. 향이 없어요. 평점1 21.08.26. 옵션 선택: 09_[1+1] 새와나무 디퓨저 / 디퓨저 향: 소형/망고쟈스민X2+스틱 / 꽃 스틱: 새와나무X2+쇼핑백(소)1P 

38 내돈내산 물건치고 최악입니다.;;어떻게 택배 물건 관리를 이렇게 하실수가 있는거죠..??ㅠㅠ
포장이 뜯겨져 있는거 알면서도 물건발송하는건 아닌가 싶어요..아무리 무배이고 저렴한 물건이라도 이렇게 해서는 안된다고 봅니다~
택배물건 받아보구 이렇게 까지 어처구니없고 황당한 기분은 처음이네요...ㅠㅠ
싼게 비지떡이라해도..이건 아니다 싶어요~~
다른사람들도 더같은 기분 들게 하지마시고 신경써서 발송하시길 바랍니다~~ 평점1 22.04.02. 선택1: 01_CO200/블랙체리 / 선택2: 04_CO200/프렌치라벤더 / 데코스틱: 04_유칼립AX2+쇼핑백/소 

39 내용물에 비해 박스가 너무 커서
박스 열어보고는 깨지진 않았나 염려스러웠어요.

개봉 후 화장실과 옷방에 두었는데 이유 없는 머리 아픔이 생겼어요 ㅠㅠ 혹시나 해서 디퓨져 치웠더니 두통 없어졌오요 ㅠㅠ.
아쉽지만 폐기합니다. ㅠㅠ 평점1 21.01.19. 옵션 스틱선택: 05_[1+1]유칼립투스 디퓨저 / 디퓨저 향선택1: 소

페이지 6 

85 생각보다 향이 저렴하더라고요~~ 싼거라 어쩔 수 없나봐요. 재구매 의사 없습니다. 평점1 20.12.26. 옵션 스틱선택: 09_[1+1]수국 디퓨저 / 디퓨저 향선택1: 소형/퓨어코튼X2+스틱/블랙X2 / 디퓨저 향선택2: 수국/베이지AX2 

86 진짜 이 향 사지마세요 머리 아픔 냄새 너무 별로
디퓨저는 무조건 직접 향 맡아보고 사야겠다는 생각이 들게한 디퓨저임 평점1 21.05.23. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

87 포장이 잘 되지 않아 안에 구성품이 파손되어 배송 되었네요. 평점1 21.02.09. 옵션 선택: 03_[1+1]수국 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 수국/베이지AX2+명절깁트백(대)1P 

88 대형마트에서 사서 향이 너무 좋아 인터넷 구매하였는데 향 차이가 너무 나네요.. 극 실망입니다. 평점1 21.02.23. 옵션 스틱선택: 03_[1+1]목화솜 디퓨저 / 디퓨저 향선택1: 소형/플라워블라썸X2+스틱/블랙X2 / 디퓨저 향선택2: 목화솜/조화X2 

89 다른 제품을 보냈으면 미리 말을 해야지 보낸 상자에 품절이라 다른거 보냈다고 써놓고 얼버무리는게 말이되나요? 평점1 20.12.24. 옵션 스틱선택: 01_[1+1]새와나무 디퓨저 / 디퓨저 향선택1: 소형/블랙체리X2 / 디퓨저 향선택2: 우드스틱/새와나무X2 

90 향 진짜별로예요...
너무 공중화장실 냄새같음.... 둘다.. 평점1 21.10.07. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 03_소형/퓨어코튼+스틱5P / 꽃 스틱: 02_수국/베이지X2+쇼핑백/소 

91 사진이랑 꽃이 너무 다르네요. 심지어 부서져서왔어요. 평점1 21.02.23. 옵션 선택: 07_[1+1]스윗플라워 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 국화/파스텔블루X2+쇼핑백(

154 유리관이 깨져서옴.... 평점1 20.11.06. 옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 05_소형/플라워블로썸+스틱5PX2 / 디퓨저 향선택2: 05_소형/플라워블로썸 

155 향기 잘 안나는데영.. 평점1 20.08.09. 디퓨저 향선택: 소형/블랙체리X2 / 꽃스틱 선택: 02_수국/베이지X2 / 리드스틱 선택: 섬유스틱5P/블랙X2 

156 향이 너무 않나요.,.. 평점1 21.10.11. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

페이지 10 

157 후기가 좋아서 주문했는데 냄새가 너무독해요 ㅜ 시트러스라벤더는 그나마 괜찮은데 에이프릴은 너무 독해요 ㅠ 평점1 22.07.01. 선택1: 04_CO200/프렌치라벤더 / 선택2: 07_CO200/플라워블로썸 / 데코스틱: 03_수국/베이지AX2+쇼핑백/소 

158 냄새 별루네요 ㅠㅠ. 평점1 21.01.02. 옵션 스틱선택: 09_[1+1]수국 디퓨저 / 디퓨저 향선택1: 소형/에이프릴X2+스틱/블랙X2 / 디퓨저 향선택2: 수국/베이지AX2 

159 냄새가 거의 안나요. 평점1 21.05.07. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

160 향기 병에 코박아야 냄새 나네요 거실에 2개 놓았는데도 향기 없음 ㅎㅎ 요즘 디퓨져 살게 없네요 평점1 21.08.12. 옵션 선택: 07_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 장미꽃/핑크X2+쇼핑백(소)1P 

161 냄새 너무 역하네요 평점1 21.08.21. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

162 향이 전혀 안나네요 평점1 21.08.25. 옵션 선택: 01_[

215 서비스 제품 최악
차량용 방향제는 냄새 하나도 안남. 일주일 이상 기다리라고만 함. 그래도 참고 참았는데 전혀 방향 효과 없고 방향제 소진만 되고 있음. 평점1 21.10.08. 옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙X2 / 꽃 스틱: 야자잎스틱X2+쇼핑백(소)1P 

216 발향이 너무 약해요;;
진짜 향이 거의 안나요 바로 옆에 가야만 향 맡을수 있어요..
리필도 두개나 샀는데 넘 화나네요 평점1 21.11.01. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 04_소형/플라워블로썸+스틱5P / 꽃 스틱: 02_수국/베이지X2+쇼핑백/소 

페이지 13 

217 로즈퍼퓸은 반품된걸 보냈나요? 상자입구쪽에 찢어져있고 해바라기는 뭔가 지저분합니다. 해바라기 잎들이 많이 접혀있어서 펴도 안펴지네요. 그냥 버렸고요....휴 평점1 22.11.16. 선택1: 04_CO200/프렌치라벤더 / 선택2: 07_CO200/로즈퍼퓸 / 선택3: 01_장미꽃/핑크X2+쇼핑백/소 

218 택배상자 테이핑 너무 허술함.
뭐 장난해놓은수준이고
결국엔 리필용2개 분실됨. 관리사무실에 cctv확인해서 신고하긴 할거지만, 짜증나는 상황이네요. 뭔 상자 테이핑을 그런식으로하는지.
향은 써오던거라 아는향이고. 평점1 21.10.12. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 03_소형/퓨어코튼+스틱5P / 꽃 스틱: 06_라벤더2PX2+쇼핑백/소 

219 크리스마스 분위기 나면서블랙체리향이 온집아세 은은히 퍼진다고 지인 선물ㅋ 평점1 21.12.21. 선택1: 01_CO200/블랙체리 / 선택2: 01_CO200/블랙체리 / 데코스틱: 01_루돌프X2+목도리/초록X2+쇼핑백/소 

220 예전엔 이 향기가 아니었는데, 실망이네요... 버리고 다른 제품 다시 주문합니다 평점1 22.06.03. 선택1: 04_CO200/퓨어코튼 / 선택2: 04_CO200/퓨어코튼 / 데코스틱: 04_유칼립AX2+

275 별1개도아까워요 소비자를뭘로보고 평점1 22.01.26. 선택1: 06_CO200/플로랄부케 / 선택2: 08_CO200/잉글리쉬 / 데코스틱: 01_뉴이어스틱X2+리본/골드X2+명절/기박 

276 좋아요잘쓸게요 라벤더구입했어요 평점1 21.01.15. 옵션 스틱선택: 07_[1+1]라벤더 디퓨저 / 디퓨저 향선택1: 소형/에이프릴X2+스틱/블랙X2 / 디퓨저 향선택2: 라벤더/퍼플/2pX2 

277 고급진 향기 절대 아니고 두통유발 평점1 21.05.21. 옵션 선택: 02_[1+1]유칼립투스 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙X2 / 꽃 스틱: 유칼립AX2+쇼핑백(소)1P 

278 봉투가 쫌 작아요ㅠㅠ 찐득찐득하구여ㅠㅜ 손상이 많이 된 봉투...... 선물용인데 맘이 좋지는 않네요.. 평점1 21.06.24. 옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블랙X2 / 꽃 스틱: 야자잎스틱X2+쇼핑백(소)1P 

279 향기가 너무약하네요 거의 안나요 평점1 21.12.13. 디퓨저 향1: 01_소형/블랙체리+스틱/5P / 디퓨저 향2: 01_소형/블랙체리+스틱5P / 꽃 스틱: 03_비누장미/소/진핑크X4+쇼핑백/소 

280 냄새가 너머 싸구려고 머리아파요 평점1 21.10.24. 디퓨저 향1: 02_소형/에이프릴+스틱/5P / 디퓨저 향2: 03_소형/퓨어코튼+스틱5P / 꽃 스틱: 01_동백꽃/옐로우X2+쇼핑백/소 

281 향선택을잘못해서너무별로에요ㅜ 평점1 21.07.29. 옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/플라워블라썸X2+스틱/블랙X2 / 꽃 스틱: 라벤더/퍼플/2pX2+쇼핑백(소)1P 

282 냄새가 싼티나요 화장실에놨어요 평점1 21.10.24. 디퓨저 향1: 05_소형/망고쟈스민+스틱/5P / 디퓨저 향2: 02_소형/에이프릴+스틱5P / 꽃 스틱: 05_비누장미/소/화이트X4+쇼핑백/소 

283 별로 토나와 밖에 버림 머리아프 평점1 20.12.

In [30]:
# 1점 리뷰 데이터프레임으로 표현
df5=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df5.head()

,type,category,review,star,date,option
1,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,선택1: 04_CO200/프렌치라벤더 / 선택2: 10_CO200/로즈퍼퓸 / 데코...
2,코코도르,별점,별 1개도아깝네요 배송 이따위로보내고 소비자 기만합니까??!! 재고 떨이하기에 급급...,평점1,22.01.26.,선택1: 05_CO200/리프레싱에어 / 선택2: 11_CO200/화이트머스크 / ...
3,코코도르,별점,분명 퓨어코튼을 시켰는데 블랙체리가 오는 신기한 마법 좋아요.블랙체리 한번 사보고 ...,평점1,21.06.30.,옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...
4,코코도르,별점,"그냥 네이버검색해서 구매했는데요. 글쎄요 상품평과 달리 향이 진하게 나지도않고, 향...",평점1,20.11.01.,옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 01_소형/블랙체리+스...
5,코코도르,별점,블랙체리향이 아닙니다. 처음 딱 느껴지는 향은 풍선껌향인가? 싶어서 당황했네요. 좀...,평점1,21.04.13.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙...


In [31]:
# 전처리 후 크롤링한 1점 데이터의 갯수
len(df5)

332

In [32]:
# 1점 데이터 전처리

df5 = df5.drop_duplicates()
df5 = df5.reset_index(drop=True)
df5_5 = df5['option'].str.split('/',expand=True)
df5 = pd.concat([df5,df5_5],axis=1)
df5.head()

,type,category,review,star,date,option,0,1,2,3,4,5,6,7,8
0,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,선택1: 04_CO200/프렌치라벤더 / 선택2: 10_CO200/로즈퍼퓸 / 데코...,선택1: 04_CO200,프렌치라벤더,선택2: 10_CO200,로즈퍼퓸,데코스틱: 01_루돌프X2+목도리,초록X2+쇼핑백,소,None,None
1,코코도르,별점,별 1개도아깝네요 배송 이따위로보내고 소비자 기만합니까??!! 재고 떨이하기에 급급...,평점1,22.01.26.,선택1: 05_CO200/리프레싱에어 / 선택2: 11_CO200/화이트머스크 / ...,선택1: 05_CO200,리프레싱에어,선택2: 11_CO200,화이트머스크,데코스틱: 06_자작나무X2+명절,기박,None,None,None
2,코코도르,별점,분명 퓨어코튼을 시켰는데 블랙체리가 오는 신기한 마법 좋아요.블랙체리 한번 사보고 ...,평점1,21.06.30.,옵션 선택: 04_[1+1]야자잎 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...,옵션 선택: 04_[1+1]야자잎 디퓨저,디퓨저 향: 소형,퓨어코튼X2+스틱,블랙X2,꽃 스틱: 야자잎스틱X2+쇼핑백(소)1P,None,None,None,None
3,코코도르,별점,"그냥 네이버검색해서 구매했는데요. 글쎄요 상품평과 달리 향이 진하게 나지도않고, 향...",평점1,20.11.01.,옵션 스틱선택: 01_수국/베이지AX2 / 디퓨저 향선택1: 01_소형/블랙체리+스...,옵션 스틱선택: 01_수국,베이지AX2,디퓨저 향선택1: 01_소형,블랙체리+스틱5PX2,디퓨저 향선택2: 03_소형,망고쟈스민,None,None,None
4,코코도르,별점,블랙체리향이 아닙니다. 처음 딱 느껴지는 향은 풍선껌향인가? 싶어서 당황했네요. 좀...,평점1,21.04.13.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/블랙체리X2+스틱/블랙...,옵션 선택: 06_[1+1]장미 디퓨저,디퓨저 향: 소형,블랙체리X2+스틱,블랙X2,꽃 스틱: 장미꽃,핑크2겹기본X2+쇼핑백(소)1P,None,None,None


In [33]:
# 전처리 후 크롤링한 1점 데이터의 갯수
len(df5)

308

In [34]:
# 5점~1점 데이터합치기
df_ascent = pd.concat([df1,df2,df3,df4,df5])
df_ascent.head()

,type,category,review,star,date,option,0,1,2,3,4,5,6,7,8
0,코코도르,별점,박스 아래 에어하나 깔아서 보낸거라 유리제품 배송인거치고는 포장이 꼼꼼한 편은 아니...,평점4,20.11.11.,옵션 스틱선택: 06_[1+1]그린트리 디퓨저 / 디퓨저 향선택1: 소형/망고쟈스민...,옵션 스틱선택: 06_[1+1]그린트리 디퓨저,디퓨저 향선택1: 소형,망고쟈스민X2+스틱,블랙X2,디퓨저 향선택2: 그린트리스틱X2,None,None,None,None
1,코코도르,별점,생애 디퓨저를 처음 구매해봅니다\n필요성을 못느끼다가 이번에 이사짐정리하다\n방에서...,평점5,21.03.20.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...,옵션 선택: 06_[1+1]장미 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 장미꽃,핑크2겹기본X2+쇼핑백(소)1P,None,None,None
2,코코도르,별점,딸아이 친구가 이사가면서 딸냄을 초대했어요. 그래도 이사 후 초대받은거여서 그냥 보...,평점5,21.06.03.,옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...,옵션 선택: 01_[1+1]라벤더 디퓨저,디퓨저 향: 소형,퓨어코튼X2+스틱,블랙X2,꽃 스틱: 라벤더,퍼플,2pX2+쇼핑백(소)1P,None,None
3,코코도르,별점,향이 나지 않습니다. 디퓨저를 사용하는 가장 중요한 목적인 '향' 부분이 빵점에 가...,평점2,21.03.20.,옵션 선택: 06_[1+1]장미 디퓨저 / 디퓨저 향: 소형/에이프릴X2+스틱/블랙...,옵션 선택: 06_[1+1]장미 디퓨저,디퓨저 향: 소형,에이프릴X2+스틱,블랙X2,꽃 스틱: 장미꽃,핑크2겹기본X2+쇼핑백(소)1P,None,None,None
4,코코도르,별점,생애 디퓨저를 처음 구매해봅니다\n필요성을 못느끼다가 이번에 이사짐정리하다\n방에서...,평점5,21.06.03.,옵션 선택: 01_[1+1]라벤더 디퓨저 / 디퓨저 향: 소형/퓨어코튼X2+스틱/블...,옵션 선택: 01_[1+1]라벤더 디퓨저,디퓨저 향: 소형,퓨어코튼X2+스틱,블랙X2,꽃 스틱: 라벤더,퍼플,2pX2+쇼핑백(소)1P,None,None


In [35]:
# 1점~5점 데이터의 총 갯수
len(df_ascent)

4859

In [36]:
### CSV파일로 저장
df_ascent.to_csv('코코도르_평점별리뷰.csv')